<a href="https://colab.research.google.com/github/sajimenezs/Proyecto-Final-Compiladores/blob/main/Proyecto_Final_Compiladores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compilador para Go - Proyecto Final de Compiladores

# Introducción



Go es un lenguaje de programación multiparadigma el cual es considerado por algunos progamadores como la evolución de C y C++, y cuya síntaxis está basada en C, pero que a diferencia de este último, no posee aritmética de punteros ni manejo de excepciones. En este lenguaje, colocar un ; al final de cada instrucción es opcional. Al igual que C, Go es un lenguaje compilado.

En el siguiente trabajo se diseña un compilador para Go, partiendo por una gramática libre de contexto(GLC) que genera su lenguaje y después, usando dicha gramática para construir un parser para dicho lenguaje

Un compilador front para Golang implementado con Python Lex y YACC (PLY) que maneja las construcciones de interruptor y bucle de Golang. Para esto usamos PLY el cual es una implementación de herramientas de análisis sintáctico **lex** y **yacc** para Python. PLY proporciona la mayoría de las características estándar de lex/yacc, incluyendo soporte para producciones vacías, reglas de precedencia, recuperación de errores y soporte para gramáticas ambiguas.

# Preliminar

En el siguiente código definimos las clases necesarias manipular el código de entrada y el árbol obtenido:

In [ ]:
class Code:
    """Sea crea una clase Code que guarda cualquier trozo de código dado"""

    def __init__(self):
        self.code = []

    def add_line(self, line):
        self.code.append(line)

    def print_code(self):
        for i in range(len(self.code)):
            print(self.code[i])

    def length(self):
        return len(self.code)

class ThreeAddressCode(Code):
    """Extendemos la clase Code para almacenar el 3AC"""

    def __init__(self):
        Code.__init__(self)
        self.leaders = []

    def add_leader(self, leader):
        if leader not in self.leaders:
            self.leaders.append(leader)

    def append_TAC(self, tac):
        for i in range(tac.length()):
            self.add_line(tac.code[i])
        self.leaders += tac.leaders

class TreeNode:
    """ Se define a clase que nos permitirá más adelante construir un árbol de derivación para una expresión p"""

    def __init__(self, name, data, input_type, isLvalue = None, children = None, TAC = None):
        self.name = name
        self.data = data
        self.input_type = input_type

        if isLvalue is None:
            self.isLvalue = 0
        else:
            self.isLvalue = isLvalue

        if children is None:
            self.children = []
        else:
            self.children = children

        if TAC is None:
            self.TAC = ThreeAddressCode()
        else:
            self.TAC = TAC

    def print_node(self):
        import time
        print("Name:", self.name, "Data:", self.data, "Type:", self.input_type, "Is L-Value?", self.isLvalue)
        if self.TAC.length() != 0:
            print("Three Address Code: ")
            self.TAC.print_code()

# Análisis Léxico



Empezamos definiendo los elementos léxicos que nos permitan tokenizar:

### Comentarios:

Los comentarios sirven como documentación del programa. Existen dos formas:

Los comentarios de línea comienzan con la secuencia de caracteres // y se detienen al final de la línea.
Los comentarios generales comienzan con la secuencia de caracteres /* y terminan con la primera secuencia de caracteres subsiguiente */.
Un comentario no puede empezar dentro de una runa o cadena literal, ni dentro de un comentario. Un comentario general que no contenga nuevas líneas actúa como un espacio. Cualquier otro comentario actúa como una nueva línea.

### Identificadores:

Los identificadores nombran entidades del programa, como las variables y los tipos. Un identificador es una secuencia de una o más letras y dígitos. El primer carácter de un identificador debe ser una letra.

    identifier = letter { letter | unicode_digit }


### Palabras clave:

Las siguientes palabras clave están reservadas y no pueden utilizarse como identificadores:

    break        default      func         interface    select
    case         defer        go           map          struct
    chan         else         goto         package      switch
    const        fallthrough  if           range        type
    continue     for          import       return       var

### Operadores y puntuación:

Las siguientes secuencias de caracteres representan operadores (incluyendo operadores de asignación) y signos de puntuación:

    +    &     +=    &=     &&    ==    !=    (    )
    -    |     -=    |=     ||    <     <=    [    ]
    *    ^     *=    ^=     <-    >     >=    {    }
    /    <<    /=    <<=    ++    =     :=    ,    ;
    %    >>    %=    >>=    --    !     ...   .    :
         &^          &^=          ~

### Literales de enteros:

Un literal entero es una secuencia de dígitos que representa una constante entera. Un prefijo opcional establece una base no decimal: 0b o 0B para el binario, 0, 0o, o 0O para el octal, y 0x o 0X para el hexadecimal. Un solo 0 se considera un cero decimal. En los literales hexadecimales, las letras de la a a la f y de la A a la F representan los valores del 10 al 15.

Para facilitar la lectura, puede aparecer un carácter de subrayado _ después de un prefijo de base o entre dígitos sucesivos; estos subrayados no cambian el valor del literal.

    int_lit = decimal_lit | binary_lit | octal_lit | hex_lit .
    lit_decimal = "0" | ( "1" ... "9" ) [ [ "_" ] decimal_digits ] .
    binary_lit = "0" ( "b" | "B" ) [ "_" ] binary_digits .
    octal_lit = "0" [ "o" | "O" ] [ "_" ] octal_digits .
    hex_lit = "0" ( "x" | "X" ) [ "_" ] hex_digits .

    decimal_digits = decimal_digit { [ "_" ] decimal_digit } .
    binary_digits = binary_digit { [ "_" ] binary_digit } .
    octal_digits = octal_digit { [ "_" ] octal_digit } .
    hex_digits = hex_digit { [ "_" ] hex_digit } .


### Literales de coma flotante

Un literal de punto flotante es una representación decimal o hexadecimal de una constante de punto flotante.

Un literal decimal de punto flotante consta de una parte entera (dígitos decimales), un punto decimal, una parte fraccionaria (dígitos decimales) y una parte de exponente (e o E seguido de un signo opcional y dígitos decimales). Se puede eludir una de las partes enteras o la parte fraccionaria; se puede eludir una de las partes decimales o la parte del exponente. Un valor de exponente exp escala la mantisa (parte entera y fraccionaria) en 10exp.

Un literal de punto flotante hexadecimal consiste en un prefijo 0x o 0X, una parte entera (dígitos hexadecimales), un punto radial, una parte fraccionaria (dígitos hexadecimales) y una parte de exponente (p o P seguido de un signo opcional y dígitos decimales). Se puede eludir una de las partes enteras o la parte fraccionaria; también se puede eludir el punto radial, pero la parte del exponente es obligatoria. (Esta sintaxis coincide con la indicada en IEEE 754-2008 §5.12.3.) Un valor de exponente exp escala la mantisa (parte entera y fraccionaria) en 2exp.

Para facilitar la lectura, puede aparecer un carácter de subrayado _ después de un prefijo de base o entre dígitos sucesivos; estos subrayados no cambian el valor literal.

    float_lit         = decimal_float_lit | hex_float_lit .

    decimal_float_lit = decimal_digits "." [ decimal_digits ] [ decimal_exponent ] |
                        decimal_digits decimal_exponent |
                        "." decimal_digits [ decimal_exponent ] .
    decimal_exponent  = ( "e" | "E" ) [ "+" | "-" ] decimal_digits .

    hex_float_lit     = "0" ( "x" | "X" ) hex_mantissa hex_exponent .
    hex_mantissa      = [ "_" ] hex_digits "." [ hex_digits ] |
                        [ "_" ] hex_digits |
                        "." hex_digits .
    hex_exponent      = ( "p" | "P" ) [ "+" | "-" ] decimal_digits .


### Literales de cadena o texto


Un literal de cadena representa una constante de cadena obtenida al concatenar una secuencia de caracteres. Existen dos formas: literales de cadena crudos y literales de cadena interpretados.

Los literales de cadena sin procesar son secuencias de caracteres entre comillas, como en \`foo\`. Dentro de las comillas, puede aparecer cualquier carácter, excepto las comillas traseras. El valor de un literal de cadena sin interpretar es la cadena compuesta por los caracteres no interpretados (codificados implícitamente en UTF-8) entre las comillas; en particular, las barras invertidas no tienen un significado especial y la cadena puede contener nuevas líneas. Los caracteres de retorno de carro ('\r') dentro de los literales de cadena sin interpretar se descartan del valor de la cadena sin interpretar.

Los literales de cadena interpretados son secuencias de caracteres entre comillas dobles, como en "bar". Dentro de las comillas, puede aparecer cualquier carácter, excepto la nueva línea y las comillas dobles sin mayúsculas. El texto entre las comillas forma el valor del literal, con escapes de barra invertida interpretados como en los literales rúnicos (excepto que \' es ilegal y \" es legal), con las mismas restricciones. Los escapes octales de tres dígitos (\nnn) y hexadecimales de dos dígitos (\xnn) representan bytes individuales de la cadena resultante; todos los demás escapes representan la codificación UTF-8 (posiblemente multibyte) de caracteres individuales. Así, dentro de una cadena literal, \377 y \xFF representan un único byte de valor 0xFF=255, mientras que ÿ, \u00FF, \U000000FF y \xc3\xbf representan los dos bytes 0xc3 0xbf de la codificación UTF-8 del carácter U+00FF.

    string_lit             = raw_string_lit | interpreted_string_lit .
    raw_string_lit         = "`" { unicode_char | newline } "`" .
    interpreted_string_lit = `"` { unicode_value | byte_value } `"` .



Con el siguiente código en el cual usamos la librería "ply.lex" construimos el con lo anterior el tokenizador.

In [ ]:
import ply.lex as lex
import sys
import os

semimode = False

def setSemiMode():
    global semimode
    if input_code[lexer.lexpos] == '\n':
        semimode = True

# Las siguientes son las palabras reservadas en GoLang
reserved_keywords = {
    'nil'      : 'NIL',
    'default'  : 'DEFAULT',
    'func'     : 'FUNC',
    'select'   : 'SELECT',
    'case'     : 'CASE',
    'go'       : 'GO',
    'struct'   : 'STRUCT',
    'else'     : 'ELSE',
    'goto'     : 'GOTO',
    'package'  : 'PACKAGE',
    'switch'   : 'SWITCH',
    'const'    : 'CONST',
    'if'       : 'IF',
    'type'     : 'TYPE',
    'for'      : 'FOR',
    'import'   : 'IMPORT',
    'var'      : 'VAR',
    'true'     : 'TRUE',
    'false'    : 'FALSE'
}

# These are the list of all tokens which we will be going to use in our building of lexer
tokens = list(reserved_keywords.values()) + [
    'PLUS',
    'MINUS',
    'STAR',
    'DIVIDE',
    'MODULO',
    'AMP',
    'OR',
    'CARET',
    'LS',
    'RS',
    'AND_OR',
    'PLUS_EQ',
    'MINUS_EQ',
    'STAR_EQ',
    'DIVIDE_EQ',
    'MODULO_EQ',
    'AMP_EQ',
    'OR_EQ',
    'CARET_EQ',
    'LS_EQ',
    'RS_EQ',
    'AND_OR_EQ',
    'AMP_AMP',
    'OR_OR',
    'LT_MINUS',
    'PLUS_PLUS',
    'MINUS_MINUS',
    'EQ_EQ',
    'LT',
    'GT',
    'EQ',
    'NOT',
    'NOT_EQ',
    'LT_EQ',
    'GT_EQ',
    'ASSIGN_OP',
    'LSQUARE',
    'RSQUARE',
    'LROUND',
    'RROUND',
    'LCURLY',
    'RCURLY',
    'COMMA',
    'DDD',
    'DOT',
    'SEMICOLON',
    'COLON',
    'SINGLE_QUOTES',
    'DOUBLE_QUOTES',

    'DECIMAL_LIT',
    'OCTAL_LIT',
    'HEX_LIT',
    'FLOAT_LIT',
    'STRING_LIT',

    # 'UNICODE_DIGIT','UNICODE_LETTER',
    # 'ESCAPED_CHAR', 'BYTE_VALUE', 'OCTAL_BYTE_VALUE', 'HEX_BYTE_VALUE',
    # 'UNDERSCORE'
    'NEWLINE',
    'IDENTIFIER',
    'PREDEF_TYPE',
    'BREAK',
    'CONTINUE',
    'RETURN', 
    'invalidIdent'
]

t_ignore = ' \t'

def t_COMMENT(t):
    r'(/\*([^*]|\n|(\*+([^*/]|\n])))*\*+/)|(//.*)'
    pass

def t_BREAK(t):
    r'break'
    setSemiMode()
    return t

def t_CONTINUE(t):
    r'continue'
    setSemiMode()
    return t

def t_RETURN(t):
    r'return'
    setSemiMode()
    return t

# Note: We need to have tokens in such a way that
# the tokens like '==' should preceede the token '='
def t_PLUS_PLUS(t):
    r'(\+\+)'
    setSemiMode()
    return t

def t_MINUS_MINUS(t):
    r'(--)'
    setSemiMode()
    return t

t_LS_EQ = r'(<<=)'
t_RS_EQ = r'(>>=)'
t_AND_OR_EQ = r'(&\^=)'
t_LS = r'(<<)'
t_RS = r'(>>)'
t_AND_OR = r'&\^'
t_PLUS_EQ = r'(\+=)'
t_MINUS_EQ = r'(-=)'
t_STAR_EQ = r'(\*=)'
t_DIVIDE_EQ = r'/='
t_MODULO_EQ = r'(%=)'
t_AMP_EQ = r'(&=)'
t_OR_EQ = r'(\|=)'
t_CARET_EQ = r'(\^=)'
t_AMP_AMP = r'(&&)'
t_OR_OR = r'(\|\|)'
t_LT_MINUS  = r'(<-)'
t_EQ_EQ = r'(==)'
t_NOT_EQ = r'(!=)'
t_NOT = r'!'
t_LT_EQ = r'(<=)'
t_GT_EQ = r'(>=)'
t_ASSIGN_OP = r'(:=)'
t_LSQUARE = r'\['
t_LROUND = r'\('
t_LCURLY = r'\{'
t_COMMA = r'\,'
t_DDD = r'\.\.\.'
t_DOT = r'\.'
t_SEMICOLON = r'\;'
t_COLON = r'\:'
t_DOUBLE_QUOTES = r'\"'
t_SINGLE_QUOTES = r'\''
t_PLUS = r'\+'
t_MINUS = r'-'
t_EQ = r'='
t_LT = r'<'
t_GT = r'>'
t_AMP = r'\&'
t_STAR = r'\*'
t_DIVIDE = r'\/'
t_MODULO = r'\%'
t_OR = r'\|'
t_CARET = r'\^'

def t_invalidIdent(t):
    r'[0-9][a-zA-Z_]+'
    print(f"LEXICAL ERROR: Invalid Identifier '{t.value}' on line {t.lexer.lineno}")
    sys.exit(-1)

def t_PREDEF_TYPE(t):
    r'((int)|(float)|(char)|(string)|(bool))'
    return t

def t_HEX_LIT(t):
    r'0[x|X][0-9A-Fa-f]+'
    setSemiMode()
    return t

def t_FLOAT_LIT(t):
    r'([0-9]+\.([0-9]+)?((e|E)(\+|\-)?[0-9]+)?)|([0-9]+(e|E)(\+|\-)?[0-9]+)|(\.[0-9]+((e|E)(\+|\-)?[0-9]+)?)'
    setSemiMode()
    return t

def t_OCTAL_LIT(t):
    r'0[0-7]*'
    setSemiMode()
    return t

def t_DECIMAL_LIT(t):
    r'[1-9][0-9]*'
    setSemiMode()
    return t

def t_RCURLY(t):
    r'\}'
    setSemiMode()
    return t

def t_RROUND(t):
    r'\)'
    setSemiMode()
    return t

def t_RSQUARE(t):
    r'\]'
    setSemiMode()
    return t

def t_STRING_LIT(t):
    r'(\"[^(\")]*\")|(\`[^(\`)]*\`)'
    t.value = t.value[1:-1]
    setSemiMode()
    return t

def t_NEWLINE(t):
    r'\n+'
    t.lexer.lineno += len(t.value)
    global semimode
    if semimode:
        semimode = False
        o = lex.LexToken()
        o.type = 'SEMICOLON'
        o.value = ';'
        o.lineno = t.lexer.lineno
        o.lexpos = t.lexer.lexpos
        return o

def t_IDENTIFIER(t):
    r'[a-zA-Z_@][a-zA-Z_0-9]*'
    t.type = reserved_keywords.get(t.value, 'IDENTIFIER')
    setSemiMode()
    return t

def t_error(t):
    print("There is an illegal character '%s' in the input program" % t.value[0])
    t.lexer.skip(1)

if len(sys.argv) < 2:
    print("Please provide a valid file path!")
    sys.exit(-1)
input_file = sys.argv[1]
if os.path.isfile(input_file) is False:
    print('Input file ' + input_file + ' does not exist')
    sys.exit(1)

input_code = open(input_file, 'r').read()
if input_code[len(input_code)-1] != '\n':
    input_code += '\n'
lexer = lex.lex()
lexer.input(input_code)
token_stream = []

t = lexer.token()
while t is not None:
    token_stream.append(t)
    t = lexer.token()
'''
for token in token_stream:
    print(token)
'''

# Análisis sintáctico

La siguiente gramática genera el lenguaje Go. Este lenguaje es aceptado por YACC, luego es de tipo LALR(1), y por lo tanto, puede ser usado más adelante para crear una Tabla de parser para el lenguaje.

    #@title Default title text
    SourceFile : PACKAGE IDENTIFIER SEMICOLON ImportDeclList TopLevelDeclList
        
    ImportDeclList : ImportDecl SEMICOLON ImportDeclList
                          | empty
        
    TopLevelDeclList : TopLevelDecl SEMICOLON TopLevelDeclList
                            | empty
        
    TopLevelDecl : Declaration
                        | FunctionDecl
        
    ImportDecl : IMPORT LROUND ImportSpecList RROUND
                      | IMPORT ImportSpec
        
    ImportSpecList : ImportSpec SEMICOLON ImportSpecList
                          | empty
        
    ImportSpec : DOT string_lit
                      | IDENTIFIER string_lit
                      | empty string_lit
        
    Block : LCURLY ScopeStart StatementList ScopeEnd RCURLY
        
    ScopeStart : empty
        
    ScopeEnd : empty
        
    StatementList : Statement SEMICOLON StatementList
                        | empty
        
    Statement : Declaration
                    | SimpleStmt
                    | ReturnStmt
                    | Block
                    | IfStmt
                    | SwitchStmt
                    | ForStmt
                    | PrintIntStmt
                    | PrintStrStmt
        
    PrintIntStmt : PRINTLN LROUND IDENTIFIER RROUND
                        | PRINTLN LROUND int_lit RROUND
        
    PrintStrStmt : PRINTLN LROUND string_lit RROUND
        
    Declaration : ConstDecl
                      | TypeDecl
                      | VarDecl
        
    ConstDecl : CONST LROUND ConstSpecList RROUND
                    | CONST ConstSpec
        
    ConstSpecList : empty
                        | ConstSpecList ConstSpec SEMICOLON
        
    ConstSpec : IDENTIFIER
                    | IdentifierList
                    | IDENTIFIER EQ Expression
                    | IdentifierList EQ ExpressionList
                    | IDENTIFIER Type EQ Expression
                    | IdentifierList Type EQ ExpressionList
        
    IdentifierList : IDENTIFIER COMMA IdentifierBotList
        
    IdentifierBotList : IDENTIFIER COMMA IdentifierBotList
                            | IDENTIFIER
        
    ExpressionList : Expression COMMA ExpressionBotList
        
    ExpressionBotList : Expression COMMA ExpressionBotList
                            | Expression
        
    Type : StandardTypes
            | LROUND Type RROUND
        
    StandardTypes : PREDEFINED_TYPES
        
    FunctionType : FUNC Signature
        
    Signature : Parameters
                    | Parameters Result
        
    Result : Parameters
                  | Type
        
    Parameters : LROUND RROUND
                      | LROUND ParameterList RROUND
        
    ParameterList : ParameterDecl
                        | ParameterList COMMA ParameterDecl
        
    ParameterDecl : IdentifierList Type
                        | IDENTIFIER Type
                        | Type
        
    VarDecl : VAR VarSpecTopList
        
    VarSpecTopList : VarSpec
                          | LROUND VarSpecList RROUND
        
    VarSpecList : empty
                      | VarSpecList VarSpec SEMICOLON
        
    VarSpec : IDENTIFIER Type
                  | IDENTIFIER EQ Expression
                  | IDENTIFIER Type EQ Expression
                  | IdentifierList Type
                  | IdentifierList EQ ExpressionList
                  | IdentifierList Type EQ ExpressionList
        
    FunctionDecl : FUNC FunctionName Signature
                        | FUNC FunctionName Signature FunctionBody
        
    FunctionName : IDENTIFIER
        
    FunctionBody : Block
        
    SimpleStmt : Expression
                      | Assignment
                      | ShortVarDecl
                      | IncDecStmt
        
    IncDecStmt : Expression PLUS_PLUS
                      | Expression MINUS_MINUS
        
    ShortVarDecl : ExpressionList ASSIGN_OP ExpressionList
                        | Expression ASSIGN_OP Expression
        
    Assignment : ExpressionList assign_op ExpressionList
                      | Expression assign_op Expression
        
    assign_op : EQ
                    | PLUS_EQ
                    | MINUS_EQ
                    | OR_EQ
                    | CARET_EQ
                    | STAR_EQ
                    | DIVIDE_EQ
                    | MODULO_EQ
                    | LS_EQ
                    | RS_EQ
                    | AMP_EQ
                    | AND_OR_EQ
        
    IfStmt : IF Expression Block
                  | IF Expression Block ELSE elseTail
        
    elseTail : IfStmt
                    | Block
        
    SwitchStmt : ExprSwitchStmt
        
    ExprSwitchStmt : SWITCH SimpleStmt SEMICOLON LCURLY ScopeStart ExprCaseClauseList ScopeEnd RCURLY
                          | SWITCH SimpleStmt SEMICOLON Expression LCURLY ScopeStart ExprCaseClauseList ScopeEnd RCURLY
                          | SWITCH LCURLY ScopeStart ExprCaseClauseList ScopeEnd RCURLY
                          | SWITCH Expression LCURLY ScopeStart ExprCaseClauseList ScopeEnd RCURLY
        
    ExprCaseClauseList : empty
                              | ExprCaseClauseList ExprCaseClause
        
    ExprCaseClause : ExprSwitchCase COLON StatementList
        
    ExprSwitchCase : CASE ExpressionList
                          | DEFAULT
                          | CASE Expression
        
    ForStmt : FOR Expression Block
                  | FOR Block
        
    ReturnStmt : RETURN
                      | RETURN Expression
                      | RETURN ExpressionList
        
    Expression : UnaryExpr
                      | Expression OR_OR Expression
                      | Expression AMP_AMP Expression
                      | Expression EQ_EQ Expression
                      | Expression NOT_EQ Expression
                      | Expression LT Expression
                      | Expression LT_EQ Expression
                      | Expression GT Expression
                      | Expression GT_EQ Expression
                      | Expression PLUS Expression
                      | Expression MINUS Expression
                      | Expression OR Expression
                      | Expression CARET Expression
                      | Expression STAR Expression
                      | Expression DIVIDE Expression
                      | Expression MODULO Expression
                      | Expression LS Expression
                      | Expression RS Expression
                      | Expression AMP Expression
                      | Expression AND_OR Expression
        
    UnaryExpr : PrimaryExpr
                    | unary_op UnaryExpr
        
    unary_op : PLUS
                    | MINUS
                    | NOT
                    | CARET
                    | STAR
                    | AMP
                    | LT_MINUS
        
    PrimaryExpr : Operand
                      | IDENTIFIER
                      | PrimaryExpr Selector
                      | PrimaryExpr Index
                      | PrimaryExpr Arguments
        
    Operand  : Literal
                    | LROUND Expression RROUND
        
    Literal  : BasicLit
                    | FunctionLit
        
    BasicLit : int_lit
                    | float_lit
                    | string_lit
                    | rune_lit
        
    int_lit : decimal_lit
                  | octal_lit
                  | hex_lit
        
    decimal_lit : DECIMAL_LIT
        
    float_lit : FLOAT_LIT
        
    FunctionLit : FUNC Signature FunctionBody
        
    Selector : DOT IDENTIFIER
        
    Index : LSQUARE Expression RSQUARE
        
    Arguments : LROUND RROUND
                    | LROUND ExpressionList RROUND
                    | LROUND Expression RROUND
                    | LROUND Type RROUND
                    | LROUND Type COMMA ExpressionList RROUND
                    | LROUND Type COMMA Expression RROUND
        
    string_lit : STRING_LIT

## Análisis léxico

In [ ]:
from symboltable import SymbolTable
from code import ThreeAddressCode

start = 0


def get_all_stmt_lhs(three_addr_code, name):
    stmts = []
    for idx, line in enumerate(three_addr_code.code):
        if len(line) == 4 and line[1] == name:
            stmts.append(idx)
    return stmts


def get_all_assignments(three_addr_code, name):
    stmts = []
    for idx, line in enumerate(three_addr_code.code):
        if line[0] in ['=', ':='] and line[2] == name and line[-1] == '':
            stmts.append(idx)
    return stmts


def pack_temps(symbol_table, three_addr_code):
    '''
    Remove all extra temporaries
    '''
    code = three_addr_code.code
    remove = list()
    for row in symbol_table.symbol_table:
        #print("FOR", row.name)
        stmts = get_all_stmt_lhs(three_addr_code, row.name)
        assignments = get_all_assignments(three_addr_code, row.name)
        try:
            if row.name.startswith("temp_"):
                #print(f"PACKING {three_addr_code.code[stmts[0]]}, {three_addr_code.code[assignments[0]]}")
                final_name = code[assignments[0]][1]
                three_addr_code.code[stmts[0]][1] = final_name
                remove.append(assignments[0])
        except:
            #print("COULD NOT PACK", stmts, assignments)
            continue
    n = len(three_addr_code.code)
    three_addr_code.code = [three_addr_code.code[i]
                            for i in range(n) if i not in remove]
    return three_addr_code

def check_line_expr(line):
    l1_flag = line[0] not in ["goto", "break", "continue"]
    l2_flag = len(line) == 4
    return l1_flag or l2_flag


def common_subexpr_eliminate(three_addr_code):
    '''
    Remove duplicate lines from Three Address Code
    Duplicates occur due to common subexpressions being repeated
    '''
    c = three_addr_code.code
    to_remove = list()
    for line in c:
        line_count = c.count(line)
        if line_count > 1:
            indices = [i for i, value in enumerate(c) if (value == line and value[0] in ['+', '-', '*', '/', ':=', '='])]
            to_remove += indices[1:]
    to_remove = list(set(to_remove))
    three_addr_code.code = [c[i] for i in range(len(c)) if i not in to_remove]
    return three_addr_code


def constant_folding(three_addr_code: ThreeAddressCode, symbol_table: SymbolTable):
    to_remove = list()
    for idx, line in enumerate(three_addr_code.code):
        if len(line) == 4:
            node1 = symbol_table.search_node(line[2])
            node2 = symbol_table.search_node(line[3])
            if node1:
                three_addr_code.code[idx][2] = node1.value
                to_remove += get_all_stmt_lhs(three_addr_code, node1.name)
            if node2:
                three_addr_code.code[idx][3] = node2.value
                to_remove += get_all_stmt_lhs(three_addr_code, node2.name)
    three_addr_code.code = [three_addr_code.code[i] for i in range(
        len(three_addr_code.code)) if i not in to_remove]
    return three_addr_code


def optimize_tac(symbol_table, three_addr_code):
    '''
    Function that performs optimizations on TAC
    '''
    three_addr_code = common_subexpr_eliminate(three_addr_code)
    print("\nAFTER COMMON SUBEXPRESSION ELIMINATION:")
    three_addr_code.print_code()
    three_addr_code = constant_folding(three_addr_code, symbol_table)
    #print("Code length: ", len(three_addr_code.code))
    print("\nAFTER CONSTANT FOLDING:")
    three_addr_code.print_code()
    #print("Code length: ", len(three_addr_code.code))
    return three_addr_code

También, definimos una tabla de símbolos para el lenguaje, usando una clase:

In [ ]:
class symboltable_node:
    """Defines a class Code which stores any piece of code line by line"""

    def __init__(self):
        """Initializes empty list named code"""
        self.name = ''
        self.type = ''
        self.scope = 0
        self.value = 0
        self.expr = None
        self.exprnode = None
    # def add_line(self, line):
    #     """Appends a line to the code"""
    #     self.code.append(line)

    # def print_code(self):
    #     """Prints the code line by line"""
    #     for i in range(len(self.code)):
    #         print(self.code[i])

    # def length(self):
    #     """Returns length of code"""
    #     return len(self.code)


class SymbolTable:
    """Defines a class for p which stores the element for the Node"""

    def __init__(self):
        """Initializes class TreeNode"""

        self.symbol_table = []

    def print_symbol_table(self):
        """Prints the symbol table"""
        print('\nSYMBOL TABLE')
        print("NAME  |  TYPE  |  SCOPE  |  VALUE  |  EXPR\n--------------------------------------------------")
        for i in range(len(self.symbol_table)):
            entry = self.symbol_table[i]
            print(entry.name, "\t", entry.type, "\t",
                  entry.scope, "\t ", entry.value, "\t   ", entry.expr)
            '''
            if entry.name.startswith("temp") or entry.name.startswith("label"):
                pass
            else:
                print(entry.name, "\t", entry.type, "\t", entry.scope)
            '''

    def add_node(self, symboltable_node):
        name = symboltable_node.name
        for st_node in self.symbol_table:
            if st_node.name == name:
                return False
        self.symbol_table.append(symboltable_node)
        return True

    def search_node(self, name):
        for i in range(len(self.symbol_table)):
            if self.symbol_table[i].name == name:
                return self.symbol_table[i]
        return []
    
    def search_expr(self, expr):
        for i in range(len(self.symbol_table)):
            if self.symbol_table[i].expr == expr:
                return self.symbol_table[i]
        return []
    

#Parser

A continuación, usando el analizador léxico descrito anteriormente, podremos construir un parser para el lenguaje, el cual decidirá si una entrada dada cumple con las reglas de la gramática construida anteriormente y en caso afirmativo, construirá un árbol de derivación para la misma y nos permitirá ejecutar la instrucción descrita

In [ ]:
import ply.lex as lex
import ply.yacc as yacc
from lexer import tokens

from code import TreeNode
from code import ThreeAddressCode
from symboltable import SymbolTable
from symboltable import symboltable_node
#import codegen
import optimize_tac
import sys
from random import *
import logging
import re

parsed = []
current_scope = 0
temp_counter = 0
label_counter = 0

ThreeAddrCode = ThreeAddressCode()
SymbolTable = SymbolTable()


def tempGen():
    global temp_counter
    tempname = 'temp_' + str(temp_counter)
    temp_counter += 1
    return tempname


def labelGen():
    global label_counter
    labelname = 'label_' + str(label_counter)
    label_counter += 1
    return labelname


def insertType(value):
    float_regex = re.compile(
        r'([0-9]+\.([0-9]+)?((e|E)(\+|\-)?[0-9]+)?)|([0-9]+(e|E)(\+|\-)?[0-9]+)|(\.[0-9]+((e|E)(\+|\-)?[0-9]+)?)')
    dec_regex = re.compile(r'[1-9][0-9]*')
    str_regex = re.compile(r'(\"[^(\")]*\")|(\`[^(\`)]*\`)')
    if value == '0':
        return 'INT'
    elif float_regex.match(value):
        return 'FLOAT'
    elif dec_regex.match(value):
        return 'INT'
    elif str_regex.match(value):
        return 'STR'
    else:
        return 'EXPR'


def getValue(name):
    symtab_entry = SymbolTable.search_node(name)
    if symtab_entry:
        vartype = symtab_entry.type
        val = symtab_entry.value
        if vartype == 'INT':
            return int(val)
        elif vartype == 'FLOAT':
            return float(val)
        else:
            return val
    else:
        try:
            val = name
            typ = insertType(name)
            if typ == 'INT':
                return int(val)
            elif typ == 'FLOAT':
                return float(val)
            else:
                return val
        except:
            return "NULL"


def evalExpr(op1, op, op2):
    v1 = getValue(op1.data)
    v2 = getValue(op2.data)
    if op == '+':
        return v1 + v2
    elif op == '-':
        return v1 - v2
    elif op == '*':
        return v1 * v2
    elif op == '/':
        return v1 / v2
    elif op == '<':
        return int(v1 < v2)
    elif op == '>':
        return int(v1 > v2)
    elif op == '<=':
        return int(v1 <= v2)
    elif op == '>=':
        return int(v1 >= v2)
    elif op == '==':
        return int(v1 == v2)
    
#Orden de prioridad de los tokens
precedence = (
    ('left', 'IDENTIFIER'),
    ('right', 'ASSIGN_OP'),
    ('left', 'COMMA'),
    ('left', 'LSQUARE'),
    ('left', 'RSQUARE'),
    ('left', 'LCURLY'),
    ('left', 'RCURLY'),
    ('left', 'DDD'),
    ('left', 'DOT'),
    ('left', 'SEMICOLON'),
    ('left', 'COLON'),
    ('left', 'SINGLE_QUOTES'),
    ('left', 'DOUBLE_QUOTES'),
    ('left', 'DECIMAL_LIT'),
    ('left', 'OCTAL_LIT'),
    ('left', 'HEX_LIT'),
    ('left', 'FLOAT_LIT'),
    ('left', 'STRING_LIT'),
    ('left', 'NEWLINE'),
    ('left', 'BREAK'),
    ('left', 'CONTINUE'),
    ('left', 'RETURN'),
    ('left', 'RROUND'),
    ('left', 'LROUND'),
    ('left', 'OR_OR'),
    ('left', 'AMP_AMP'),
    ('left', 'EQ_EQ', 'NOT_EQ', 'LT', 'LT_EQ', 'GT', 'GT_EQ'),
    ('left', 'PLUS', 'MINUS', 'OR', 'CARET'),
    ('left', 'STAR', 'DIVIDE', 'MODULO', 'AMP', 'AND_OR', 'LS', 'RS'),
)


def p_SourceFile(p):
    '''SourceFile : PACKAGE IDENTIFIER SEMICOLON ImportDeclList TopLevelDeclList
    '''
    # TODO: Ignoring package name and Imports for now
    p[0] = p[5]
    print("Code length: ", len(p[0].TAC.code))
    p[0].TAC.print_code()
    print("\n\nOPTMIZATION\n\n")
    tac_optimized = optimize_tac.optimize_tac(SymbolTable, p[0].TAC)
    print("\nSYMBOL TABLE:")
    SymbolTable.print_symbol_table()


def p_ImportDeclList(p):
    '''ImportDeclList : ImportDecl SEMICOLON ImportDeclList
                | empty
    '''
    # TODO: Ignoring Imports for now
    p[0] = p[1]
    return
    # parsed.append(p.slice)


def p_TopLevelDeclList(p):
    '''TopLevelDeclList : TopLevelDecl SEMICOLON TopLevelDeclList
                        | empty
    '''
    if len(p) == 4:
        if p[3] != None:
            p[0] = TreeNode('TopLevelDeclList', 0, 'INT', 0,
                            [p[1]] + p[3].children, p[1].TAC)
            p[0].TAC.append_TAC(p[3].TAC)
        else:
            p[0] = TreeNode('TopLevelDeclList', 0, 'INT', 0, [p[1]], p[1].TAC)
    return


def p_TopLevelDecl(p):
    '''TopLevelDecl  : Declaration
                    | FunctionDecl
    '''
    p[0] = p[1]
    # p[0].print_node()
    return


def p_ImportDecl(p):
    '''ImportDecl : IMPORT LROUND ImportSpecList RROUND
                    | IMPORT ImportSpec
    '''
    # TODO: Ignoring Imports for now
    return
    # parsed.append(p.slice)


def p_ImportSpecList(p):
    '''ImportSpecList : ImportSpec SEMICOLON ImportSpecList
                        | empty
    '''
    # TODO: Ignoring Imports for now
    return
    # parsed.append(p.slice)


def p_ImportSpec(p):
    '''ImportSpec :  DOT string_lit
                    | IDENTIFIER string_lit
                    | empty string_lit
    '''
    # TODO: Ignoring Imports for now
    return
    # parsed.append(p.slice)


def p_Block(p):
    '''Block : LCURLY ScopeStart StatementList ScopeEnd RCURLY
    '''
    p[0] = p[3]
    p[0].name = 'Block'
    return
    # parsed.append(p.slice)


def p_ScopeStart(p):
    '''ScopeStart : empty
    '''
    global current_scope
    current_scope += 1
    return


def p_ScopeEnd(p):
    '''ScopeEnd : empty
    '''
    global current_scope
    current_scope -= 1
    return


def p_StatementList(p):
    '''StatementList : Statement SEMICOLON StatementList
                    | empty
    '''
    if len(p) == 4:
        p[0] = TreeNode('StatementList', 0, 'INT', 0, [
                        p[1].data] + p[3].children, p[1].TAC)
        p[0].TAC.append_TAC(p[3].TAC)
    else:
        p[0] = TreeNode('StatementList', 0, 'INT')
    # p[0].print_node()


def p_Statement(p):
    '''Statement : Declaration
                 | SimpleStmt
                 | ReturnStmt
                 | Block
                 | IfStmt
                 | SwitchStmt
                 | ForStmt
                 | BreakStmt
                 | ContinueStmt
                 | GotoStmt
    '''
    #print(p.slice)
    p[0] = p[1]
    p[0].name = 'Statement'


def p_Declaration(p):
    '''Declaration  : ConstDecl
                 | TypeDecl
                 | VarDecl
    '''
    parsed.append(p.slice)
    p[0] = p[1]
    p[0].name = 'Declaration'


def p_ConstDecl(p):
    '''ConstDecl : CONST LROUND ConstSpecList RROUND
                 | CONST ConstSpec
                 | CONST IDENTIFIER
    '''
    parsed.append(p.slice)


def p_ConstSpecList(p):
    '''ConstSpecList : empty
                 | ConstSpecList ConstSpec SEMICOLON
                 | ConstSpecList IDENTIFIER SEMICOLON
    '''
    parsed.append(p.slice)


def p_ConstSpec(p):
    '''ConstSpec : IdentifierList
                 | IdentifierList Type EQ Expression
                 | IDENTIFIER Type EQ Expression
                 | IdentifierList Type EQ ExpressionList
                 | IDENTIFIER Type EQ ExpressionList
                 | IdentifierList IDENTIFIER DOT IDENTIFIER EQ Expression
                 | IDENTIFIER IDENTIFIER DOT IDENTIFIER EQ Expression
                 | IdentifierList IDENTIFIER DOT IDENTIFIER EQ ExpressionList
                 | IDENTIFIER IDENTIFIER DOT IDENTIFIER EQ ExpressionList
                 | IdentifierList IDENTIFIER EQ Expression
                 | IDENTIFIER IDENTIFIER EQ Expression
                 | IdentifierList IDENTIFIER EQ ExpressionList
                 | IDENTIFIER IDENTIFIER EQ ExpressionList
                 | IdentifierList EQ Expression
                 | IDENTIFIER EQ Expression
                 | IdentifierList EQ ExpressionList
                 | IDENTIFIER EQ ExpressionList
    '''

    parsed.append(p.slice)


def p_IdentifierList(p):
    '''IdentifierList : IDENTIFIER COMMA IdentifierBotList
    '''
    p[0] = TreeNode('IdentifierList', 0, 'INT', 0, [p[1]] + p[3].children)
    return


def p_IdentifierBotList(p):
    '''IdentifierBotList : IDENTIFIER COMMA IdentifierBotList
                 | IDENTIFIER
    '''
    if len(p) == 2:
        p[0] = TreeNode('IdentifierBotList', p[1], 'INT')
        return
    elif len(p) == 4:
        p[0] = TreeNode('IdentifierBotList', 0, 'INT',
                        0, [p[1]] + p[3].children)
        return


def p_ExpressionList(p):
    '''ExpressionList : Expression COMMA ExpressionBotList
    '''
    p[0] = TreeNode('ExpressionList', 0, 'INT', [p[1].isLvalue] +
                    p[3].isLvalue, [p[1]] + p[3].children, p[1].TAC)
    p[0].TAC.append_TAC(p[3].TAC)
    return


def p_ExpressionBotList(p):
    '''ExpressionBotList : Expression COMMA ExpressionBotList
                        | Expression
    '''
    if len(p) == 2:
        p[0] = TreeNode('ExpressionBotList', 0, 'INT', [
                        p[1].isLvalue], [p[1]], p[1].TAC)
        return
    elif len(p) == 4:
        p[0] = TreeNode('ExpressionBotList', 0, 'INT', [
                        p[1].isLvalue] + p[3].isLvalue, [p[1]] + p[3].children, p[1].TAC)
        p[0].TAC.append_TAC(p[3].TAC)
        return


def p_TypeDecl(p):
    '''TypeDecl : TYPE TypeSpecTopList
    '''
    parsed.append(p.slice)


def p_TypeSpecTopList(p):
    '''TypeSpecTopList : TypeSpec
                 | LROUND TypeSpecList  RROUND
    '''
    parsed.append(p.slice)


def p_TypeSpecList(p):
    '''TypeSpecList : empty
                 | TypeSpecList TypeSpec SEMICOLON
    '''
    parsed.append(p.slice)


def p_TypeSpec(p):
    '''TypeSpec : AliasDecl
                 | TypeDef
    '''
    parsed.append(p.slice)


def p_AliasDecl(p):
    '''AliasDecl : IDENTIFIER EQ Type
                 | IDENTIFIER EQ IDENTIFIER DOT IDENTIFIER
                 | IDENTIFIER EQ IDENTIFIER
    '''
    parsed.append(p.slice)


def p_TypeDef(p):
    '''TypeDef : IDENTIFIER Type
                 | IDENTIFIER IDENTIFIER
                 | IDENTIFIER IDENTIFIER DOT IDENTIFIER
    '''
    parsed.append(p.slice)


def p_Type(p):
    '''Type :  TypeLit
                 | LROUND IDENTIFIER RROUND
                 | LROUND Type RROUND
                 | LROUND IDENTIFIER DOT IDENTIFIER RROUND
                 | empty
    '''
    parsed.append(p.slice)
    if len(p) == 2:
        #print("ADDED ", p[1])
        p[0] = p[1]
    else:
        p[0] = p[2]
    p[0].name = 'Type'


def p_TypeLit(p):
    '''TypeLit : ArrayType
                 | StandardType
                 | StructType
                 | FunctionType
    '''
    p[0] = p[1]
    parsed.append(p.slice)

def p_StandardType(p):
    '''StandardType : PREDEF_TYPE
    '''
    parsed.append(p.slice) 
    #print("PREDEF_TYPE", p[1])
    p[0] = TreeNode('StandardTypes', p[1], 'NONE')
    return

def p_ArrayType(p):
    '''ArrayType : LSQUARE ArrayLength RSQUARE Type
                 | LSQUARE ArrayLength RSQUARE IDENTIFIER
                 | LSQUARE ArrayLength RSQUARE IDENTIFIER DOT IDENTIFIER
    '''
    parsed.append(p.slice)


def p_ArrayLength(p):
    '''ArrayLength : Expression
    '''
    parsed.append(p.slice)


def p_StructType(p):
    '''StructType : STRUCT LCURLY FieldDeclList RCURLY
    '''
    parsed.append(p.slice)


def p_FieldDeclList(p):
    '''FieldDeclList : empty
                 | FieldDeclList FieldDecl SEMICOLON
    '''
    parsed.append(p.slice)


def p_FieldDecl(p):
    '''FieldDecl : IdentifierList Type TagTop
                 | IdentifierList IDENTIFIER TagTop
                 | IDENTIFIER IDENTIFIER
                 | IdentifierList IDENTIFIER DOT IDENTIFIER TagTop
                 | STAR IDENTIFIER DOT IDENTIFIER TagTop
                 | IDENTIFIER DOT IDENTIFIER TagTop
                 | STAR IDENTIFIER TagTop
                 | IDENTIFIER TagTop
    '''
    parsed.append(p.slice)


def p_TagTop(p):
    '''TagTop : empty
                 | Tag
    '''
    parsed.append(p.slice)


def p_Tag(p):
    '''Tag : string_lit
    '''
    parsed.append(p.slice)


def p_FunctionType(p):
    '''FunctionType : FUNC Signature
    '''
    parsed.append(p.slice)

# Signature      = Parameters [ Result ] .
# Result         = Parameters | Type .
# Parameters     = "(" [ ParameterList [ "," ] ] ")" .
# ParameterList  = ParameterDecl { "," ParameterDecl } .
# ParameterDecl  = [ IdentifierList ] [ "..." ] Type .


def p_Signature(p):
    '''Signature : Parameters
                 | Parameters Result
    '''
    parsed.append(p.slice)


def p_Result(p):
    '''Result : Parameters
                 | Type
                 | IDENTIFIER
                 | IDENTIFIER DOT IDENTIFIER
    '''
    parsed.append(p.slice)


def p_Parameters(p):
    '''Parameters : LROUND RROUND
                 | LROUND ParameterList RROUND
                 | LROUND ParameterList COMMA RROUND
    '''
    parsed.append(p.slice)


def p_ParameterList(p):
    '''ParameterList  : ParameterDecl
                 | ParameterList COMMA ParameterDecl
    '''
    parsed.append(p.slice)


def p_ParameterDecl(p):
    '''ParameterDecl  : DDD Type
                 | IdentifierList Type
                 | IdentifierList DDD Type
                 | IDENTIFIER Type
                 | IDENTIFIER DDD Type
                 | DDD IDENTIFIER
                 | IdentifierList IDENTIFIER
                 | IdentifierList DDD IDENTIFIER
                 | IDENTIFIER IDENTIFIER
                 | IDENTIFIER DDD IDENTIFIER
                 | DDD IDENTIFIER DOT IDENTIFIER
                 | IdentifierList IDENTIFIER DOT IDENTIFIER
                 | IdentifierList DDD IDENTIFIER DOT IDENTIFIER
                 | IDENTIFIER IDENTIFIER DOT IDENTIFIER
                 | IDENTIFIER DDD IDENTIFIER DOT IDENTIFIER
    '''
    parsed.append(p.slice)


def p_VarDecl(p):
    '''VarDecl : VAR VarSpecTopList
    '''
    parsed.append(p.slice)
    p[0] = p[2]
    p[0].name = 'VarDecl'
    return


def p_VarSpecTopList(p):
    '''VarSpecTopList : VarSpec
                      | LROUND VarSpecList RROUND
    '''
    parsed.append(p.slice)
    if len(p) == 2:
        p[0] = p[1]
    else:
        p[0] = p[2]
    p[0].name = 'VarSpecTopList'
    return


def p_VarSpecList(p):
    '''VarSpecList : empty
                 | VarSpecList VarSpec SEMICOLON
    '''

    parsed.append(p.slice)


def p_VarSpec(p):
    '''VarSpec : IDENTIFIER Type
               | IDENTIFIER EQ Expression
               | IDENTIFIER Type EQ Expression
               | IdentifierList Type
               | IdentifierList EQ ExpressionList
               | IdentifierList Type EQ ExpressionList
    '''
    # Insert into symbol table
    p[0] = TreeNode('VarSpec', 0, 'NONE')
    if hasattr(p[1], 'name') and  p[1].name == 'IdentifierList':
        zero_val = TreeNode('decimal_lit', 0, 'INT')
    else:
        p[1] = TreeNode('IDENTIFIER',p[1],'INT',1)
        if p[2].input_type != 'NONE':
            node = symboltable_node()
            node.name = p[1].data
            node.value = None
            node.scope = current_scope
            node.type = p[1].input_type
            SymbolTable.add_node(node)
        p[0] = TreeNode('VarSpec',p[1].data,'INT')
    return


def p_VarSpecMid(p):
    '''VarSpecMid : empty
                 | EQ ExpressionList
                 | EQ Expression
    '''
    parsed.append(p.slice)


def p_FunctionDecl(p):
    '''FunctionDecl : FUNC FunctionName Type Signature
                | FUNC FunctionName Signature
                | FUNC FunctionName Type Signature FunctionBody
                | FUNC FunctionName Signature FunctionBody
    '''
    p[0] = TreeNode('FunctionDecl', 0, 'INT')
    p[0].TAC.add_line(['func', p[2].data, '', ''])
    if len(p) == 5:
        p[0].TAC.append_TAC(p[4].TAC)
    # p[0].print_node()
    return
    # p[2].print_node()
    # p[4].print_node()
    # parsed.append(p.slice)


def p_FunctionName(p):
    '''FunctionName : IDENTIFIER
    '''
    p[0] = TreeNode('FunctionName', p[1], 'INT')
    return


def p_FunctionBody(p):
    '''FunctionBody : Block
    '''
    p[0] = p[1]
    p[0].name = 'FunctionBody'
    return


def p_SimpleStmt(p):
    '''SimpleStmt : Expression
                 | Assignment
                 | ShortVarDecl
                 | IncDecStmt
    '''
    p[0] = p[1]
    p[0].name = 'SimpleStmt'
    return
    # parsed.append(p.slice)

# def p_ExpressionStmt(p):
#     '''ExpressionStmt : Expression
#     '''
#     parsed.append(p.slice)


def p_IncDecStmt(p):
    '''IncDecStmt : Expression PLUS_PLUS
                | Expression MINUS_MINUS
    '''
    one_val = TreeNode('decimal_lit', 1, 'INT')
    p[0] = p[1]
    if p[1].isLvalue == 1:
        if p[2] == '++':
            p[0].TAC.add_line(['+', p[1].data, p[1].data, one_val.data])
        else:
            p[0].TAC.add_line(['-', p[1].data, p[1].data, one_val.data])
    else:
        print("*** Error: Lvalue required! ***")
    p[0].name = 'IncDecStmt'
    return


def p_ShortVarDecl(p):
    '''ShortVarDecl : ExpressionList ASSIGN_OP ExpressionList
                 | Expression ASSIGN_OP Expression
    '''
    global current_scope
    p[0] = TreeNode('ShortVarDecl', 0, 'INT')
    if p[1].name == 'ExpressionList':
        l1 = len(p[1].children)
        l2 = len(p[3].children)
        p[0].TAC.append_TAC(p[3].TAC)
        p[0].TAC.append_TAC(p[1].TAC)
        if l1 == l2:
            for i in range(l1):
                if p[1].children[i].isLvalue == 0:
                    print("*** Error: Cannot assign to constant ***", p.lineno(1))
                else:
                    if SymbolTable.search_node(p[1].children[i].data) == []:
                        node = symboltable_node()
                        node.name = p[1].children[i].data
                        node.type = insertType(p[3].children[i].data)
                        node.scope = current_scope
                        node.value = p[3].children[i].data
                        SymbolTable.add_node(node)
                    p[0].TAC.add_line(
                        [p[2], p[1].children[i].data, p[3].children[i].data, ''])
        else:
            print("*** Error: Assignment mismatch:", l1,
                  "identifier(s) but", l2, "value(s)! ***")
            print(p.lineno(1))

    elif p[1].name == 'Expression':
        if p[1].isLvalue == 0:
            print("*** Error: Cannot declare and assign to constant ***")
            return
        else:
            p[0].TAC.append_TAC(p[3].TAC)
            p[0].TAC.append_TAC(p[1].TAC)
            p[0].TAC.add_line([p[2], p[1].data, p[3].data, ''])
            if SymbolTable.search_node(p[1].data) == []:
                node = symboltable_node()
                node.name = p[1].data
                node.type = insertType(p[3].data)
                node.scope = current_scope
                node.value = p[3].data
                SymbolTable.add_node(node)
            return


def p_Assignment(p):
    '''Assignment : ExpressionList assign_op ExpressionList
                | Expression assign_op Expression
    '''
    global current_scope
    p[0] = TreeNode('Assignment', 0, 'INT')
    if p[1].name == 'ExpressionList':
        l1 = len(p[1].children)
        l2 = len(p[3].children)
        p[0].TAC.append_TAC(p[3].TAC)
        p[0].TAC.append_TAC(p[1].TAC)
        if l1 == l2:
            for i in range(l1):
                if p[1].children[i].isLvalue == 0:
                    print("*** Error: Cannot assign to constant ***")
                else:
                    #print("Looking for: ", p[1].children[i].data)
                    if SymbolTable.search_node(p[1].children[i].data) == []:
                        node = symboltable_node()
                        node.name = p[1].children[i].data
                        node.type = insertType(p[3].children[i].data)
                        node.scope = current_scope
                        node.value = p[3].children[i].data
                        SymbolTable.add_node(node)

                    if SymbolTable.search_node(p[3].children[i].data) == [] and p[3].children[i].isLvalue == 1:
                        #print("Looking for: ", p[3].children[i].data)
                        node = symboltable_node()
                        node.name = p[3].children[i].data
                        node.type = insertType(p[3].children[i].data)
                        node.scope = current_scope
                        node.value = p[3].children[i].data
                        SymbolTable.add_node(node)
                    p[0].TAC.add_line(
                        [p[2].data, p[1].children[i].data, p[3].children[i].data, ''])
        else:
            print("*** Error: Assignment mismatch:", l1,
                  "identifier(s) but", l2, "value(s)! ***")

    elif p[1].name == 'Expression':
        # p[0] = TreeNode('Assignment', 0, 'INT', 0, p[1].children + p[3].children, p[1].TAC.append_TAC(p[3].TAC))
        if p[1].isLvalue == 0:
            print("*** Error: Cannot assign to constant ***")
            return
        else:
            p[0].TAC.append_TAC(p[3].TAC)
            p[0].TAC.append_TAC(p[1].TAC)
            p[0].TAC.add_line([p[2].data, p[1].data, p[3].data, ''])
            # and p[1].children[i].isLvalue ==1:
            if SymbolTable.search_node(p[1].data) == []:
                print("*** Error: Variable not declared: ", p[1].data)
            if SymbolTable.search_node(p[3].data) == [] and p[3].isLvalue == 1:
                node = symboltable_node()
                node.name = p[3].data
                node.type = insertType(p[3].data)
                node.scope = current_scope
                node.value = p[3].data
                SymbolTable.add_node(node)
            return


def p_assign_op(p):
    '''assign_op : EQ
                 | PLUS_EQ
                 | MINUS_EQ
                 | OR_EQ
                 | CARET_EQ
                 | STAR_EQ
                 | DIVIDE_EQ
                 | MODULO_EQ
                 | LS_EQ
                 | RS_EQ
                 | AMP_EQ
                 | AND_OR_EQ
    '''
    p[0] = TreeNode('assign_op', p[1], 'OPERATOR')
    return


def p_IfStmt(p):
    '''IfStmt : IF Expression Block
            | IF Expression Block ELSE elseTail
    '''
    if len(p) == 4:
        l1 = labelGen()
        p[0] = TreeNode('IfStmt', 0, 'INT')
        p[0].TAC.append_TAC(p[2].TAC)
        p[0].TAC.add_line(['ifgotoeq', p[2].data, 0, l1])
        p[0].TAC.append_TAC(p[3].TAC)
        p[0].TAC.add_line([l1])
    if len(p) == 6:
        l1 = labelGen()
        l2 = labelGen()
        p[0] = TreeNode('IfStmt', 0, 'INT')
        p[0].TAC.append_TAC(p[2].TAC)
        p[0].TAC.add_line(['ifgotoeq', p[2].data, 0, l1])
        p[0].TAC.append_TAC(p[3].TAC)
        p[0].TAC.add_line(['goto', l2])
        p[0].TAC.add_line([l1])
        p[0].TAC.append_TAC(p[5].TAC)
        p[0].TAC.add_line([l2])
    # p[0].print_node()
    return


def p_elseTail(p):
    '''elseTail : IfStmt
                 | Block
    '''
    p[0] = p[1]
    p[0].name = 'elseTail'
    return


def p_SwitchStmt(p):
    '''SwitchStmt : ExprSwitchStmt
    '''
    p[0] = TreeNode('SwitchStmt', 0, 'INT', 0, [], p[1].TAC)
    return


def p_ExprSwitchStmt(p):
    '''ExprSwitchStmt : SWITCH SimpleStmt SEMICOLON LCURLY ExprCaseClauseList RCURLY
                 | SWITCH SimpleStmt SEMICOLON Expression LCURLY ExprCaseClauseList RCURLY
                 | SWITCH LCURLY ExprCaseClauseList RCURLY
                 | SWITCH Expression LCURLY ExprCaseClauseList RCURLY
    '''
    global current_scope
    if len(p) == 6:
        l1 = labelGen()
        l2 = labelGen()
        p[0] = TreeNode('ExprSwitchStmt', 0, 'INT')
        p[0].TAC.append_TAC(p[2].TAC)
        t1 = tempGen()
        node = symboltable_node()
        node.name = t1
        node.scope = current_scope
        node.value = p[2].data
        node.type = insertType(p[2].data)
        SymbolTable.add_node(node)
        p[0].TAC.add_line(['=', t1, p[2].data, ''])
        p[0].TAC.append_TAC(p[4].data)
        for i in range(len(p[4].children)):
            p[0].TAC.add_line(
                ['ifgotoeq', t1, p[4].children[i][0], p[4].children[i][1]])
        for i in range(p[4].TAC.length()):
            if i in p[4].TAC.leaders[1:]:
                p[0].TAC.add_line(['goto', l2, '', ''])
            p[0].TAC.add_line(p[4].TAC.code[i])
        p[0].TAC.add_line([l2])
    return


def p_ExprCaseClauseList(p):
    '''ExprCaseClauseList : empty
                 | ExprCaseClauseList ExprCaseClause
    '''
    TAC1 = ThreeAddressCode()
    TAC2 = ThreeAddressCode()
    if len(p) == 3:
        TAC1 = p[1].data
        TAC2 = p[2].data
        p[0] = TreeNode('ExprCaseClauseList', TAC1, 'INT', 0,
                        p[1].children + p[2].children, p[1].TAC)
        p[0].TAC.add_leader(p[0].TAC.length())
        p[0].TAC.append_TAC(p[2].TAC)
        p[0].data.append_TAC(TAC2)

    else:
        p[0] = TreeNode('ExprCaseClauseList', TAC1, 'INT')

    return
    # parsed.append(p.slice)


def p_ExprCaseClause(p):
    '''ExprCaseClause : ExprSwitchCase COLON StatementList
    '''
    l1 = labelGen()
    p[0] = TreeNode('ExprCaseClause', 0, 'INT')
    # p[0].TAC.append_TAC(p[1].TAC)
    p[0].TAC.add_line([l1])
    # p[0].TAC.add_line(['ifgotoneq', p[1].children, p[1].children, l1])
    p[0].TAC.append_TAC(p[3].TAC)
    p[0].children = [[p[1].data, l1]]
    p[0].data = p[1].TAC
    return
    # parsed.append(p.slice)


def p_ExprSwitchCase(p):
    '''ExprSwitchCase : CASE ExpressionList
                 | DEFAULT
                 | CASE Expression
    '''
    p[0] = TreeNode('ExprSwitchCase', 0, 'INT')
    if len(p) == 3:
        p[0].data = p[2].data
        p[0].TAC = p[2].TAC

    parsed.append(p.slice)


def p_ForStmt(p):
    '''ForStmt : FOR Expression Block
                 | FOR Block
    '''
    p[0] = TreeNode('ForStmt', 0, 'INT')
    if len(p) == 4:
        l1 = labelGen()
        l2 = labelGen()
        p[0].TAC.add_line([l1])
        p[0].TAC.append_TAC(p[2].TAC)
        p[0].TAC.add_line(['ifgotoeq', p[2].data, 0, l2])
        p[0].TAC.append_TAC(p[3].TAC)
        p[0].TAC.add_line(['goto', l1])
        p[0].TAC.add_line([l2])

    if len(p) == 3:
        l1 = labelGen()
        # l2 = label_gen()
        p[0].TAC.add_line([l1])
        p[0].TAC.append_TAC(p[2].TAC)
        p[0].TAC.add_line(['goto', l1])
        # p[0].TAC.add_line([l2])
    return


def p_ReturnStmt(p):
    '''ReturnStmt : RETURN
                  | RETURN Expression
                  | RETURN ExpressionList
    '''
    parsed.append(p.slice)
    if len(p) == 2:
        p[0] = TreeNode('ReturnStmt', 0, 'None')
        p[0].TAC.add_line(['return', '', '', ''])
    if len(p) == 3:
        if p[2].name == 'Expression':
            p[0] = p[2]
            p[0].name = 'ReturnStmt'
            p[0].TAC.add_line(['return', p[2].data, '', ''])
    return


def p_BreakStmt(p):
    '''BreakStmt : BREAK IDENTIFIER
    '''
    parsed.append(p.slice)


def p_ContinueStmt(p):
    '''ContinueStmt : CONTINUE IDENTIFIER
    '''
    parsed.append(p.slice)


def p_GotoStmt(p):
    '''GotoStmt : GOTO IDENTIFIER
    '''
    parsed.append(p.slice)


def p_Expression(p): 
    '''Expression : UnaryExpr
                 | Expression OR_OR Expression
                 | Expression AMP_AMP Expression
                 | Expression EQ_EQ Expression
                 | Expression NOT_EQ Expression
                 | Expression LT Expression
                 | Expression LT_EQ Expression
                 | Expression GT Expression
                 | Expression GT_EQ Expression
                 | Expression PLUS Expression
                 | Expression MINUS Expression
                 | Expression OR Expression
                 | Expression CARET Expression
                 | Expression STAR Expression
                 | Expression DIVIDE Expression
                 | Expression MODULO Expression
                 | Expression LS Expression
                 | Expression RS Expression
                 | Expression AMP Expression
                 | Expression AND_OR Expression
    '''
    global current_scope
    if len(p) == 2:
        p[0] = p[1]
    elif len(p) == 4:
        expression = p[1].data + p[2] + p[3].data
        expr_node = SymbolTable.search_expr(expression)
        if not expr_node:
            temp = tempGen()
            node = symboltable_node()
            node.name = temp
            node.value = p[1].data + p[2] + p[3].data
            node.expr = p[1].data + p[2] + p[3].data
            node.type = p[1].input_type
            node.scope = current_scope
            SymbolTable.add_node(node)
            #print(f"Evaluating expression {node.value}")
            node.value = evalExpr(p[1], p[2], p[3])
            #SymbolTable.print_symbol_table()
            #print(node.value, node.name)
            p[0] = TreeNode('IDENTIFIER', temp, 'INT', 1, [], p[1].TAC)
            node.exprnode = p[0]
            p[0].TAC.append_TAC(p[3].TAC)
            p[0].TAC.add_line([p[2], p[0].data, p[1].data, p[3].data])
        else:
            p[0] = expr_node.exprnode
    p[0].name = 'Expression'
    return


def p_UnaryExpr(p):
    '''UnaryExpr : PrimaryExpr
                 | unary_op UnaryExpr
    '''
    global current_scope
    if len(p) == 2:
        p[0] = p[1]
    elif len(p) == 3:
        temp = tempGen()
        node = symboltable_node()
        node.name = temp
        node.value = p[2].data
        node.scope = current_scope
        SymbolTable.add_node(node)
        p[0] = TreeNode('IDENTIFIER', temp, 'INT', 1)
        p[0].TAC.add_line([p[1].data, p[0].data, p[2].data])
    p[0].name = 'UnaryExpr'
    return


def p_unary_op(p):
    '''unary_op : PLUS
                 | MINUS
                 | NOT
                 | CARET
                 | STAR
                 | AMP
                 | LT_MINUS
    '''
    p[0] = TreeNode('unary_op', p[1], 'OPERATOR')
    return


def p_PrimaryExpr(p):
    '''PrimaryExpr : Operand
                 | IDENTIFIER
                 | PrimaryExpr Selector
                 | PrimaryExpr Index
                 | PrimaryExpr Arguments
    '''
    if len(p) == 2:
        if p.slice[1].type == 'IDENTIFIER':
            p[0] = TreeNode('IDENTIFIER', p[1], 'INT', 1, [])
        elif p[1].name == 'Operand':
            p[0] = p[1]
    elif len(p) == 3:
        if p[2].name == 'Arguments':
            p[0] = p[1]
            p[0].TAC.add_line(['call', p[1].data, '', ''])
    p[0].name = 'PrimaryExpr'
    return


def p_Operand(p):
    '''Operand  : Literal
                 | LROUND Expression RROUND
    '''
    if len(p) == 2:
        p[0] = p[1]
    else:
        p[0] = p[2]
    p[0].name = 'Operand'
    return


def p_Literal(p):
    '''Literal  : BasicLit
                 | FunctionLit
    '''
    if p[1].name == 'BasicLit':
        p[0] = p[1]
    elif p[1].name == 'FunctionLit':
        p[0]
        # TODO: FunctionLit
    p[0].name = 'Literal'
    return


def p_BasicLit(p):
    '''BasicLit : int_lit
                 | float_lit
                 | string_lit
    '''
    p[0] = p[1]
    p[0].name = 'BasicLit'
    return


def p_int_lit(p):
    '''int_lit : decimal_lit
                 | octal_lit
                 | hex_lit
    '''
    p[0] = p[1]
    p[0].name = 'int_lit'
    return


def p_decimal_lit(p):
    '''decimal_lit : DECIMAL_LIT
    '''
    p[0] = TreeNode('decimal_lit', p[1], 'INT')
    return


def p_octal_lit(p):
    '''octal_lit : OCTAL_LIT
    '''
    p[0] = TreeNode('octal_lit', p[1], 'OCT')
    return


def p_hex_lit(p):
    '''hex_lit  : HEX_LIT
    '''
    p[0] = TreeNode('hex_lit', p[1], 'HEX')
    return


def p_float_lit(p):
    '''float_lit : FLOAT_LIT
    '''
    p[0] = TreeNode('float_lit', p[1], 'FLOAT')
    return


def p_FunctionLit(p):
    '''FunctionLit : FUNC Signature FunctionBody
    '''
    parsed.append(p.slice)


def p_Selector(p):
    '''Selector : DOT IDENTIFIER
    '''
    parsed.append(p.slice)


def p_Index(p):
    '''Index : LSQUARE Expression RSQUARE
    '''
    parsed.append(p.slice)


def p_Arguments(p):
    '''Arguments : LROUND RROUND
                 | LROUND ExpressionList DDD RROUND
                 | LROUND Expression DDD RROUND
                 | LROUND ExpressionList RROUND
                 | LROUND Expression RROUND
                 | LROUND Type DDD RROUND
                 | LROUND Type RROUND
                 | LROUND Type COMMA ExpressionList  DDD RROUND
                 | LROUND Type COMMA ExpressionList  RROUND
                 | LROUND Type COMMA Expression DDD RROUND
                 | LROUND Type COMMA Expression RROUND
                 | LROUND IDENTIFIER DDD RROUND
                 | LROUND IDENTIFIER RROUND %prec LROUND
                 | LROUND IDENTIFIER COMMA ExpressionList  DDD RROUND
                 | LROUND IDENTIFIER COMMA ExpressionList  RROUND
                 | LROUND IDENTIFIER COMMA Expression DDD RROUND
                 | LROUND IDENTIFIER COMMA Expression RROUND
                 | LROUND IDENTIFIER DOT IDENTIFIER DDD RROUND
                 | LROUND IDENTIFIER DOT IDENTIFIER RROUND
                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA ExpressionList  DDD RROUND
                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA ExpressionList  RROUND
                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA Expression DDD RROUND
                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA Expression RROUND
                 | LROUND ExpressionList DDD COMMA RROUND
                 | LROUND Expression DDD COMMA RROUND
                 | LROUND ExpressionList COMMA RROUND
                 | LROUND Expression COMMA RROUND
                 | LROUND Type DDD COMMA RROUND
                 | LROUND Type COMMA RROUND
                 | LROUND Type COMMA ExpressionList  DDD COMMA RROUND
                 | LROUND Type COMMA ExpressionList  COMMA RROUND
                 | LROUND Type COMMA Expression DDD COMMA RROUND
                 | LROUND Type COMMA Expression COMMA RROUND
                 | LROUND IDENTIFIER DDD COMMA RROUND
                 | LROUND IDENTIFIER COMMA RROUND
                 | LROUND IDENTIFIER COMMA ExpressionList  DDD COMMA RROUND
                 | LROUND IDENTIFIER COMMA ExpressionList  COMMA RROUND
                 | LROUND IDENTIFIER COMMA Expression DDD COMMA RROUND
                 | LROUND IDENTIFIER COMMA Expression COMMA RROUND
                 | LROUND IDENTIFIER DOT IDENTIFIER DDD COMMA RROUND
                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA RROUND
                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA ExpressionList  DDD COMMA RROUND
                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA ExpressionList  COMMA RROUND
                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA Expression DDD COMMA RROUND
                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA Expression COMMA RROUND
    '''
    if len(p) == 3:
        p[0] = TreeNode('Arguments', 0, 'INT')
    return

def p_empty(p):
    'empty :'
    pass


def p_string_lit(p):
    '''string_lit : STRING_LIT
    '''
    p[0] = TreeNode('string_lit', p[1], 'STRING')
    return


def p_error(p):
    if p == None:
        print(str(sys.argv[1]) + " :: You missed something at the end")
    else:
        print(f"SYNTAX ERROR on line number {p.lineno - numLines + 1}")

A continuación se explica brevemente el funcionamiento de las funciones más relevantes del código:

- *insertType(value):* Usando una expresión regular, esta función determina si el parámetro value corresponde a un Int, a un Float, a un String, o a una Expresión.

- *getValue():* Esta función retorna el valor de una cierta variable, bien sea entera o flotante.

- *evalExpr(data1,op,data2):* Primero, obtiene el valor exacto de data 1 y data2 y después, dependiendo de qué token sea op, calcula el resultado de la operación correspondiente.

- *p_GrammarExpression(p):* De acuerdo a las producciones del símbolo no-terminal GrammarExpression, construye un nodo en la clase ClassNode para dicha derivación.

# Tabla de parser

Con lo anterior obtenemos la siguiente salida que corresponde a la tabla del Parser:

In [ ]:
# parsetab.py
# pylint: disable=W,C,R
_tabversion = '3.10'

_lr_method = 'LALR'

_lr_signature = 'leftIDENTIFIERrightASSIGN_OPleftCOMMAleftLSQUAREleftRSQUAREleftLCURLYleftRCURLYleftDDDleftDOTleftSEMICOLONleftCOLONleftSINGLE_QUOTESleftDOUBLE_QUOTESleftDECIMAL_LITleftOCTAL_LITleftHEX_LITleftFLOAT_LITleftSTRING_LITleftNEWLINEleftBREAKleftCONTINUEleftRETURNleftRROUNDleftLROUNDleftOR_ORleftAMP_AMPleftEQ_EQNOT_EQLTLT_EQGTGT_EQleftPLUSMINUSORCARETleftSTARDIVIDEMODULOAMPAND_ORLSRSAMP AMP_AMP AMP_EQ AND_OR AND_OR_EQ ASSIGN_OP BREAK CARET CARET_EQ CASE COLON COMMA CONST CONTINUE DDD DECIMAL_LIT DEFAULT DIVIDE DIVIDE_EQ DOT DOUBLE_QUOTES ELSE EQ EQ_EQ FALSE FLOAT_LIT FOR FUNC GO GOTO GT GT_EQ HEX_LIT IDENTIFIER IF IMPORT LCURLY LROUND LS LSQUARE LS_EQ LT LT_EQ LT_MINUS MINUS MINUS_EQ MINUS_MINUS MODULO MODULO_EQ NEWLINE NIL NOT NOT_EQ OCTAL_LIT OR OR_EQ OR_OR PACKAGE PLUS PLUS_EQ PLUS_PLUS PREDEF_TYPE RCURLY RETURN RROUND RS RSQUARE RS_EQ SELECT SEMICOLON SINGLE_QUOTES STAR STAR_EQ STRING_LIT STRUCT SWITCH TRUE TYPE VAR invalidIdentSourceFile : PACKAGE IDENTIFIER SEMICOLON ImportDeclList TopLevelDeclList\n    ImportDeclList : ImportDecl SEMICOLON ImportDeclList\n                | empty\n    TopLevelDeclList : TopLevelDecl SEMICOLON TopLevelDeclList\n                        | empty\n    TopLevelDecl  : Declaration\n                    | FunctionDecl\n    ImportDecl : IMPORT LROUND ImportSpecList RROUND\n                    | IMPORT ImportSpec\n    ImportSpecList : ImportSpec SEMICOLON ImportSpecList\n                        | empty\n    ImportSpec :  DOT string_lit\n                    | IDENTIFIER string_lit\n                    | empty string_lit\n    Block : LCURLY ScopeStart StatementList ScopeEnd RCURLY\n    ScopeStart : empty\n    ScopeEnd : empty\n    StatementList : Statement SEMICOLON StatementList\n                    | empty\n    Statement : Declaration\n                 | SimpleStmt\n                 | ReturnStmt\n                 | Block\n                 | IfStmt\n                 | SwitchStmt\n                 | ForStmt\n                 | BreakStmt\n                 | ContinueStmt\n                 | GotoStmt\n    Declaration  : ConstDecl\n                 | TypeDecl\n                 | VarDecl\n    ConstDecl : CONST LROUND ConstSpecList RROUND\n                 | CONST ConstSpec\n                 | CONST IDENTIFIER\n    ConstSpecList : empty\n                 | ConstSpecList ConstSpec SEMICOLON\n                 | ConstSpecList IDENTIFIER SEMICOLON\n    ConstSpec : IdentifierList\n                 | IdentifierList Type EQ Expression\n                 | IDENTIFIER Type EQ Expression\n                 | IdentifierList Type EQ ExpressionList\n                 | IDENTIFIER Type EQ ExpressionList\n\n                 | IdentifierList IDENTIFIER DOT IDENTIFIER EQ Expression\n                 | IDENTIFIER IDENTIFIER DOT IDENTIFIER EQ Expression\n                 | IdentifierList IDENTIFIER DOT IDENTIFIER EQ ExpressionList\n                 | IDENTIFIER IDENTIFIER DOT IDENTIFIER EQ ExpressionList\n\n                 | IdentifierList IDENTIFIER EQ Expression\n                 | IDENTIFIER IDENTIFIER EQ Expression\n                 | IdentifierList IDENTIFIER EQ ExpressionList\n                 | IDENTIFIER IDENTIFIER EQ ExpressionList\n                 | IdentifierList EQ Expression\n                 | IDENTIFIER EQ Expression\n                 | IdentifierList EQ ExpressionList\n                 | IDENTIFIER EQ ExpressionList\n    IdentifierList : IDENTIFIER COMMA IdentifierBotList\n    IdentifierBotList : IDENTIFIER COMMA IdentifierBotList\n                 | IDENTIFIER\n    ExpressionList : Expression COMMA ExpressionBotList\n    ExpressionBotList : Expression COMMA ExpressionBotList\n                        | Expression\n    TypeDecl : TYPE TypeSpecTopList\n    TypeSpecTopList : TypeSpec\n                 | LROUND TypeSpecList  RROUND\n    TypeSpecList : empty\n                 | TypeSpecList TypeSpec SEMICOLON\n    TypeSpec : AliasDecl\n                 | TypeDef\n    AliasDecl : IDENTIFIER EQ Type\n                 | IDENTIFIER EQ IDENTIFIER DOT IDENTIFIER\n                 | IDENTIFIER EQ IDENTIFIER\n    TypeDef : IDENTIFIER Type\n                 | IDENTIFIER IDENTIFIER\n                 | IDENTIFIER IDENTIFIER DOT IDENTIFIER\n    Type :  TypeLit\n                 | LROUND IDENTIFIER RROUND\n                 | LROUND Type RROUND\n                 | LROUND IDENTIFIER DOT IDENTIFIER RROUND\n                 | empty\n    TypeLit : ArrayType\n                 | StandardType\n                 | StructType\n                 | FunctionType\n    StandardType : PREDEF_TYPE\n    ArrayType : LSQUARE ArrayLength RSQUARE Type\n                 | LSQUARE ArrayLength RSQUARE IDENTIFIER\n                 | LSQUARE ArrayLength RSQUARE IDENTIFIER DOT IDENTIFIER\n    ArrayLength : Expression\n    StructType : STRUCT LCURLY FieldDeclList RCURLY\n    FieldDeclList : empty\n                 | FieldDeclList FieldDecl SEMICOLON\n    FieldDecl : IdentifierList Type TagTop\n                 | IdentifierList IDENTIFIER TagTop\n                 | IDENTIFIER IDENTIFIER\n                 | IdentifierList IDENTIFIER DOT IDENTIFIER TagTop\n                 | STAR IDENTIFIER DOT IDENTIFIER TagTop\n                 | IDENTIFIER DOT IDENTIFIER TagTop\n                 | STAR IDENTIFIER TagTop\n                 | IDENTIFIER TagTop\n    TagTop : empty\n                 | Tag\n    Tag : string_lit\n    FunctionType : FUNC Signature\n    Signature : Parameters\n                 | Parameters Result\n    Result : Parameters\n                 | Type\n                 | IDENTIFIER\n                 | IDENTIFIER DOT IDENTIFIER\n    Parameters : LROUND RROUND\n                 | LROUND ParameterList RROUND\n                 | LROUND ParameterList COMMA RROUND\n\n    ParameterList  : ParameterDecl\n                 | ParameterList COMMA ParameterDecl\n\n    ParameterDecl  : DDD Type\n                 | IdentifierList Type\n                 | IdentifierList DDD Type\n                 | IDENTIFIER Type\n                 | IDENTIFIER DDD Type\n                 | DDD IDENTIFIER\n                 | IdentifierList IDENTIFIER\n                 | IdentifierList DDD IDENTIFIER\n                 | IDENTIFIER IDENTIFIER\n                 | IDENTIFIER DDD IDENTIFIER\n                 | DDD IDENTIFIER DOT IDENTIFIER\n                 | IdentifierList IDENTIFIER DOT IDENTIFIER\n                 | IdentifierList DDD IDENTIFIER DOT IDENTIFIER\n                 | IDENTIFIER IDENTIFIER DOT IDENTIFIER\n                 | IDENTIFIER DDD IDENTIFIER DOT IDENTIFIER\n    VarDecl : VAR VarSpecTopList\n    VarSpecTopList : VarSpec\n                      | LROUND VarSpecList RROUND\n    VarSpecList : empty\n                 | VarSpecList VarSpec SEMICOLON\n    VarSpec : IDENTIFIER Type\n               | IDENTIFIER EQ Expression\n               | IDENTIFIER Type EQ Expression\n               | IdentifierList Type\n               | IdentifierList EQ ExpressionList\n               | IdentifierList Type EQ ExpressionList\n    VarSpecMid : empty\n                 | EQ ExpressionList\n                 | EQ Expression\n    FunctionDecl : FUNC FunctionName Type Signature\n                | FUNC FunctionName Signature\n                | FUNC FunctionName Type Signature FunctionBody\n                | FUNC FunctionName Signature FunctionBody\n    FunctionName : IDENTIFIER\n    FunctionBody : Block\n    SimpleStmt : Expression\n                 | Assignment\n                 | ShortVarDecl\n                 | IncDecStmt\n    IncDecStmt : Expression PLUS_PLUS\n                | Expression MINUS_MINUS\n    ShortVarDecl : ExpressionList ASSIGN_OP ExpressionList\n                 | Expression ASSIGN_OP Expression\n    Assignment : ExpressionList assign_op ExpressionList\n                | Expression assign_op Expression\n    assign_op : EQ\n                 | PLUS_EQ\n                 | MINUS_EQ\n                 | OR_EQ\n                 | CARET_EQ\n                 | STAR_EQ\n                 | DIVIDE_EQ\n                 | MODULO_EQ\n                 | LS_EQ\n                 | RS_EQ\n                 | AMP_EQ\n                 | AND_OR_EQ\n    IfStmt : IF Expression Block\n            | IF Expression Block ELSE elseTail\n    elseTail : IfStmt\n                 | Block\n    SwitchStmt : ExprSwitchStmt\n    ExprSwitchStmt : SWITCH SimpleStmt SEMICOLON LCURLY ExprCaseClauseList RCURLY\n                 | SWITCH SimpleStmt SEMICOLON Expression LCURLY ExprCaseClauseList RCURLY\n                 | SWITCH LCURLY ExprCaseClauseList RCURLY\n                 | SWITCH Expression LCURLY ExprCaseClauseList RCURLY\n    ExprCaseClauseList : empty\n                 | ExprCaseClauseList ExprCaseClause\n    ExprCaseClause : ExprSwitchCase COLON StatementList\n    ExprSwitchCase : CASE ExpressionList\n                 | DEFAULT\n                 | CASE Expression\n    ForStmt : FOR Expression Block\n                 | FOR Block\n    ReturnStmt : RETURN\n                  | RETURN Expression\n                  | RETURN ExpressionList\n    BreakStmt : BREAK IDENTIFIER\n    ContinueStmt : CONTINUE IDENTIFIER\n    GotoStmt : GOTO IDENTIFIER\n    Expression : UnaryExpr\n                 | Expression OR_OR Expression\n                 | Expression AMP_AMP Expression\n                 | Expression EQ_EQ Expression\n                 | Expression NOT_EQ Expression\n                 | Expression LT Expression\n                 | Expression LT_EQ Expression\n                 | Expression GT Expression\n                 | Expression GT_EQ Expression\n                 | Expression PLUS Expression\n                 | Expression MINUS Expression\n                 | Expression OR Expression\n                 | Expression CARET Expression\n                 | Expression STAR Expression\n                 | Expression DIVIDE Expression\n                 | Expression MODULO Expression\n                 | Expression LS Expression\n                 | Expression RS Expression\n                 | Expression AMP Expression\n                 | Expression AND_OR Expression\n    UnaryExpr : PrimaryExpr\n                 | unary_op UnaryExpr\n    unary_op : PLUS\n                 | MINUS\n                 | NOT\n                 | CARET\n                 | STAR\n                 | AMP\n                 | LT_MINUS\n    PrimaryExpr : Operand\n                 | IDENTIFIER\n                 | PrimaryExpr Selector\n                 | PrimaryExpr Index\n                 | PrimaryExpr Arguments\n    Operand  : Literal\n                 | LROUND Expression RROUND\n    Literal  : BasicLit\n                 | FunctionLit\n    BasicLit : int_lit\n                 | float_lit\n                 | string_lit\n    int_lit : decimal_lit\n                 | octal_lit\n                 | hex_lit\n    decimal_lit : DECIMAL_LIT\n    octal_lit : OCTAL_LIT\n    hex_lit  : HEX_LIT\n    float_lit : FLOAT_LIT\n    FunctionLit : FUNC Signature FunctionBody\n    Selector : DOT IDENTIFIER\n    Index : LSQUARE Expression RSQUARE\n    Arguments : LROUND RROUND\n                 | LROUND ExpressionList DDD RROUND\n                 | LROUND Expression DDD RROUND\n                 | LROUND ExpressionList RROUND\n                 | LROUND Expression RROUND\n                 | LROUND Type DDD RROUND\n                 | LROUND Type RROUND\n                 | LROUND Type COMMA ExpressionList  DDD RROUND\n                 | LROUND Type COMMA ExpressionList  RROUND\n                 | LROUND Type COMMA Expression DDD RROUND\n                 | LROUND Type COMMA Expression RROUND\n                 | LROUND IDENTIFIER DDD RROUND\n                 | LROUND IDENTIFIER RROUND %prec LROUND\n                 | LROUND IDENTIFIER COMMA ExpressionList  DDD RROUND\n                 | LROUND IDENTIFIER COMMA ExpressionList  RROUND\n                 | LROUND IDENTIFIER COMMA Expression DDD RROUND\n                 | LROUND IDENTIFIER COMMA Expression RROUND\n                 | LROUND IDENTIFIER DOT IDENTIFIER DDD RROUND\n                 | LROUND IDENTIFIER DOT IDENTIFIER RROUND\n                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA ExpressionList  DDD RROUND\n                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA ExpressionList  RROUND\n                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA Expression DDD RROUND\n                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA Expression RROUND\n                 | LROUND ExpressionList DDD COMMA RROUND\n                 | LROUND Expression DDD COMMA RROUND\n                 | LROUND ExpressionList COMMA RROUND\n                 | LROUND Expression COMMA RROUND\n                 | LROUND Type DDD COMMA RROUND\n                 | LROUND Type COMMA RROUND\n                 | LROUND Type COMMA ExpressionList  DDD COMMA RROUND\n                 | LROUND Type COMMA ExpressionList  COMMA RROUND\n                 | LROUND Type COMMA Expression DDD COMMA RROUND\n                 | LROUND Type COMMA Expression COMMA RROUND\n                 | LROUND IDENTIFIER DDD COMMA RROUND\n                 | LROUND IDENTIFIER COMMA RROUND\n                 | LROUND IDENTIFIER COMMA ExpressionList  DDD COMMA RROUND\n                 | LROUND IDENTIFIER COMMA ExpressionList  COMMA RROUND\n                 | LROUND IDENTIFIER COMMA Expression DDD COMMA RROUND\n                 | LROUND IDENTIFIER COMMA Expression COMMA RROUND\n                 | LROUND IDENTIFIER DOT IDENTIFIER DDD COMMA RROUND\n                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA RROUND\n                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA ExpressionList  DDD COMMA RROUND\n                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA ExpressionList  COMMA RROUND\n                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA Expression DDD COMMA RROUND\n                 | LROUND IDENTIFIER DOT IDENTIFIER COMMA Expression COMMA RROUND\n    empty :string_lit : STRING_LIT\n    '
    
_lr_action_items = {'PACKAGE':([0,],[2,]),'$end':([1,4,5,7,9,11,21,27,45,53,],[0,-291,-291,-3,-1,-5,-291,-291,-2,-4,]),'IDENTIFIER':([2,8,17,18,19,20,22,30,32,33,36,39,42,58,60,65,68,69,72,73,74,76,78,79,81,83,84,86,88,90,92,96,97,98,102,103,108,113,114,115,116,117,119,121,122,124,138,141,142,143,144,147,148,150,153,154,157,162,164,168,170,171,174,176,178,179,183,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,209,210,211,215,216,217,218,224,225,231,233,234,254,256,258,259,260,261,262,263,267,269,272,297,308,309,310,311,314,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,338,339,351,353,355,363,366,369,370,372,376,384,395,421,423,432,441,448,453,462,496,],[3,25,29,32,39,43,25,-291,70,77,-291,80,-291,97,107,110,141,-36,110,147,149,110,39,-65,158,43,-133,110,110,25,168,-291,172,-110,181,184,97,-217,-218,-220,-221,-222,110,-219,-223,110,-291,70,219,110,110,-58,-56,110,228,110,232,110,110,172,110,-16,264,265,-111,168,271,273,275,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110,295,110,302,309,-90,-37,-38,110,147,-66,316,-134,110,110,110,343,344,345,110,349,-112,352,354,357,374,375,381,110,-57,110,110,110,110,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,110,110,399,400,401,110,110,110,418,-91,422,110,110,454,456,110,110,110,110,110,110,]),'SEMICOLON':([3,6,10,12,13,14,15,16,23,31,32,33,34,35,37,38,39,40,41,43,44,47,49,50,51,52,56,57,59,60,61,62,63,64,66,80,81,82,85,87,89,91,93,94,95,98,104,105,106,107,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,139,140,141,145,146,147,148,151,152,155,156,158,159,160,161,163,165,169,173,177,178,186,206,207,208,212,220,221,222,223,226,227,229,230,232,235,236,238,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,257,267,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,306,307,308,309,312,313,314,316,322,323,336,337,342,343,344,345,346,348,350,356,359,362,365,368,373,374,375,377,378,379,380,381,382,383,385,386,387,389,390,391,392,393,394,401,402,404,405,407,408,411,413,416,419,420,422,424,425,429,435,436,437,440,443,444,447,450,452,454,455,456,457,458,459,465,466,468,469,471,472,474,475,477,478,481,483,484,485,488,489,490,491,492,495,498,499,500,502,503,505,506,507,],[4,21,27,-6,-7,-30,-31,-32,-9,-34,-35,-39,-62,-63,-67,-68,-291,-130,-131,-291,-291,90,-12,-292,-13,-14,-145,-75,-79,-104,-80,-81,-82,-83,-84,-73,-291,-72,-135,-138,-8,-103,-144,-147,-149,-110,-106,-105,-107,-108,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-33,217,218,-53,-55,-58,-56,-52,-54,-64,231,-71,-69,-132,234,-136,-139,-146,-76,-77,-111,-291,-226,-227,-228,-216,-49,-51,-41,-43,-40,-42,-48,-50,-74,-137,-140,319,-20,-21,-22,-23,-24,-25,-26,-27,-28,-29,-150,-151,-152,-153,-189,-176,-112,-109,-85,-86,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-89,372,-291,-291,-61,-59,-57,-70,-154,-155,-190,-191,-188,-192,-193,-194,395,-150,-78,-245,-249,-250,-252,-258,-291,-291,-94,-99,-100,-101,-102,-291,-45,-47,-44,-46,-15,-159,-157,-158,-156,-172,-187,-87,-247,-271,-248,-272,-251,-274,-257,-280,-92,-93,-291,-98,-60,-179,-269,-270,-273,-254,-256,-279,-260,-262,-264,-291,-97,-291,-175,-173,-174,-180,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-95,-96,-177,-275,-277,-281,-283,-285,-266,-268,-178,-288,-265,-290,-267,-287,-289,]),'IMPORT':([4,21,],[8,8,]),'FUNC':([4,5,7,21,27,28,29,32,33,39,43,44,45,58,60,65,72,74,76,81,86,88,96,97,98,102,103,108,113,114,115,116,117,119,121,122,124,141,143,144,147,148,150,154,162,164,168,170,171,176,178,183,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,267,297,308,311,314,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,338,339,363,366,369,384,395,432,441,448,453,462,496,],[-291,17,-3,-291,17,54,-148,54,54,54,54,54,-2,54,54,130,130,54,130,54,130,130,-291,54,-110,54,54,54,-217,-218,-220,-221,-222,130,-219,-223,130,54,130,130,-58,-56,130,130,130,130,54,130,-16,54,-111,54,54,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,303,130,130,130,130,130,-112,303,54,130,-57,130,130,130,130,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,130,130,130,130,130,130,130,130,130,130,130,130,130,]),'CONST':([4,5,7,21,27,45,96,170,171,319,462,],[-291,18,-3,-291,18,-2,-291,18,-16,18,18,]),'TYPE':([4,5,7,21,27,45,96,170,171,319,462,],[-291,19,-3,-291,19,-2,-291,19,-16,19,19,]),'VAR':([4,5,7,21,27,45,96,170,171,319,462,],[-291,20,-3,-291,20,-2,-291,20,-16,20,20,]),'LROUND':([8,18,19,20,28,29,32,33,39,43,44,50,54,55,57,58,59,60,61,62,63,64,65,66,72,74,76,81,86,88,91,95,96,97,98,102,103,104,105,106,107,108,110,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,141,143,144,147,148,150,154,162,164,168,170,171,173,176,177,178,183,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,210,211,224,254,256,258,262,267,273,274,275,295,297,298,302,303,304,305,306,308,311,314,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,338,339,350,356,357,359,362,363,365,366,368,369,384,387,395,401,402,404,405,407,408,411,413,416,432,435,436,437,440,441,443,444,447,448,450,452,453,462,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,496,498,500,502,503,505,506,507,],[22,30,36,42,58,-148,74,74,74,74,74,-292,92,92,-75,74,-79,108,-80,-81,-82,-83,124,-84,124,74,124,74,124,124,-103,-149,-291,74,-110,74,74,-106,-105,-107,-108,74,-225,-217,-218,-220,-221,-222,211,124,-224,-219,-223,-229,124,-231,-232,-233,-234,-235,92,-236,-237,-238,-242,-239,-240,-241,74,124,124,-58,-56,124,124,124,124,74,124,-16,-76,74,-77,-111,74,74,124,124,124,124,124,124,124,124,124,124,124,124,124,124,124,124,124,124,124,-226,-227,-228,124,297,124,124,124,124,124,-112,-109,-85,-86,-244,297,-246,-225,92,-230,-243,-89,74,124,-57,124,124,124,124,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,124,124,-78,-245,-225,-249,-250,124,-252,124,-258,124,124,-15,124,-87,-247,-271,-248,-272,-251,-274,-257,-280,124,-269,-270,-273,-254,124,-256,-279,-260,124,-262,-264,124,124,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,124,-268,-288,-265,-290,-267,-287,-289,]),'DOT':([8,22,50,70,77,80,90,95,97,107,110,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,149,158,172,181,184,206,207,208,265,271,275,295,298,302,304,305,309,356,357,359,362,365,368,374,381,387,402,404,405,407,408,411,413,416,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[24,24,-292,142,153,157,24,-149,174,185,-225,209,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,174,233,263,269,272,-226,-227,-228,351,353,355,-244,-246,370,-230,-243,376,-245,174,-249,-250,-252,-258,421,423,-15,-247,-271,-248,-272,-251,-274,-257,-280,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'STRING_LIT':([8,22,24,25,26,48,57,59,60,61,62,63,64,65,66,72,76,86,88,90,91,96,98,104,105,106,107,113,114,115,116,117,119,121,122,124,143,144,147,148,150,154,162,164,170,171,173,177,178,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,267,273,274,275,297,306,308,309,311,314,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,338,339,350,363,366,369,373,374,381,384,395,401,422,432,441,448,453,454,456,462,496,],[-291,-291,50,50,50,50,-75,-79,-104,-80,-81,-82,-83,50,-84,50,50,50,50,-291,-103,-291,-110,-106,-105,-107,-108,-217,-218,-220,-221,-222,50,-219,-223,50,50,50,-58,-56,50,50,50,50,50,-16,-76,-77,-111,-291,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,-112,-109,-85,-86,50,-89,-291,50,50,-57,50,50,50,50,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,50,50,-78,50,50,50,50,50,50,50,50,-87,50,50,50,50,50,50,50,50,50,]),'RROUND':([22,30,36,42,46,48,50,57,58,59,60,61,62,63,64,66,68,69,74,78,79,83,84,90,91,92,95,97,98,99,100,101,102,103,104,105,106,107,108,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,147,148,149,167,168,172,173,175,176,177,178,179,180,181,182,183,184,186,206,207,208,211,212,213,217,218,231,234,264,265,266,267,268,270,271,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,297,298,299,300,301,302,304,305,306,312,313,314,349,350,352,354,356,357,358,359,360,361,362,363,364,365,366,367,368,369,371,387,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,425,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,488,489,490,491,492,493,494,495,496,497,498,500,501,502,503,504,505,506,507,],[-291,-291,-291,-291,89,-11,-292,-75,98,-79,-104,-80,-81,-82,-83,-84,139,-36,-291,155,-65,160,-133,-291,-103,98,-149,173,-110,177,178,-113,-291,-291,-106,-105,-107,-108,98,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-58,-56,173,-10,-291,-123,-76,-118,-291,-77,-111,267,-115,-120,-116,-291,-121,-291,-226,-227,-228,298,-216,304,-37,-38,-66,-134,350,-124,-119,-112,-114,-117,-122,-109,-85,-86,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-291,-246,359,362,365,368,-230,-243,-89,-61,-59,-57,-128,-78,-125,-126,-245,173,402,-249,404,405,-250,407,408,-252,411,413,-258,416,-103,-15,-129,-127,-87,-247,435,-271,-248,436,-272,-251,437,440,-274,443,-257,444,447,-280,450,452,-60,-269,-270,-273,466,468,-254,469,471,-256,-279,472,474,-260,475,477,-262,478,-264,481,-276,488,-253,-278,489,-255,-282,490,-259,-284,491,-261,-263,492,495,-286,498,-275,-277,-281,-283,-285,500,502,-266,503,505,-268,-288,506,-265,-290,507,-267,-287,-289,]),'LSQUARE':([28,29,32,33,39,43,44,50,58,60,74,81,95,97,98,102,103,108,110,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,141,147,148,168,176,178,183,186,206,207,208,211,267,295,297,298,302,304,305,308,314,356,357,359,362,365,368,387,402,404,405,407,408,411,413,416,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[65,-148,65,65,65,65,65,-292,65,65,65,65,-149,65,-110,65,65,65,-225,210,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,65,-58,-56,65,65,-111,65,65,-226,-227,-228,65,-112,-244,65,-246,-225,-230,-243,65,-57,-245,-225,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'PREDEF_TYPE':([28,29,32,33,39,43,44,58,60,74,81,97,98,102,103,108,141,147,148,168,176,178,183,186,211,267,297,308,314,],[66,-148,66,66,66,66,66,66,66,66,66,66,-110,66,66,66,66,-58,-56,66,66,-111,66,66,66,-112,66,66,-57,]),'STRUCT':([28,29,32,33,39,43,44,58,60,74,81,97,98,102,103,108,141,147,148,168,176,178,183,186,211,267,297,308,314,],[67,-148,67,67,67,67,67,67,67,67,67,67,-110,67,67,67,67,-58,-56,67,67,-111,67,67,67,-112,67,67,-57,]),'EQ':([32,33,39,43,44,50,57,59,60,61,62,63,64,66,70,71,75,77,85,87,91,95,98,104,105,106,107,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,141,147,148,173,177,178,186,206,207,208,212,219,228,250,255,267,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,306,312,313,314,348,350,356,359,362,365,368,387,401,402,404,405,407,408,411,413,416,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[72,76,81,86,88,-292,-75,-79,-104,-80,-81,-82,-83,-84,143,144,150,154,162,164,-103,-149,-110,-106,-105,-107,-108,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,72,-58,-56,-76,-77,-111,-291,-226,-227,-228,-216,311,315,324,324,-112,-109,-85,-86,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-89,-61,-59,-57,324,-78,-245,-249,-250,-252,-258,-15,-87,-247,-271,-248,-272,-251,-274,-257,-280,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'COMMA':([32,43,50,57,59,60,61,62,63,64,66,91,95,97,98,100,101,102,103,104,105,106,107,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,141,145,147,148,151,166,168,172,173,175,176,177,178,180,181,182,183,184,186,206,207,208,211,212,220,222,226,229,250,265,266,267,268,270,271,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,299,300,301,302,304,305,306,309,312,313,314,336,348,349,350,352,354,356,358,359,361,362,364,365,367,368,371,382,385,387,399,400,401,402,404,405,407,408,410,411,412,413,415,416,417,418,425,435,436,437,439,440,442,443,444,446,447,449,450,451,452,464,466,468,469,471,472,474,475,477,478,480,481,482,488,489,490,491,492,494,495,497,498,500,502,503,505,506,507,],[73,73,-292,-75,-79,-104,-80,-81,-82,-83,-84,-103,-149,73,-110,179,-113,-291,-291,-106,-105,-107,-108,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,73,224,225,-56,224,224,73,-123,-76,-118,-291,-77,-111,-115,-120,-116,-291,-121,-291,-226,-227,-228,-291,-216,224,224,224,224,224,-124,-119,-112,-114,-117,-122,-109,-85,-86,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,360,363,366,369,-230,-243,-89,73,384,-59,-57,224,224,-128,-78,-125,-126,-245,403,-249,406,-250,409,-252,414,-258,-103,224,224,-15,-129,-127,-87,-247,-271,-248,-272,-251,438,-274,441,-257,445,-280,448,453,-60,-269,-270,-273,467,-254,470,-256,-279,473,-260,476,-262,479,-264,224,-276,-253,-278,-255,-282,-259,-284,-261,-263,493,-286,496,-275,-277,-281,-283,-285,501,-266,504,-268,-288,-265,-290,-267,-287,-289,]),'OR_OR':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,187,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,187,187,187,187,-226,-227,-228,-216,187,187,187,187,187,187,187,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,187,-246,187,-225,-230,-243,187,187,187,187,187,-245,-225,-249,-250,-252,-258,187,187,-15,187,187,-247,-271,-248,-272,-251,-274,187,-257,-280,187,187,-269,-270,-273,-254,-256,-279,-260,-262,-264,187,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,187,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'AMP_AMP':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,188,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,188,188,188,188,-226,-227,-228,-216,188,188,188,188,188,188,188,188,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,188,-246,188,-225,-230,-243,188,188,188,188,188,-245,-225,-249,-250,-252,-258,188,188,-15,188,188,-247,-271,-248,-272,-251,-274,188,-257,-280,188,188,-269,-270,-273,-254,-256,-279,-260,-262,-264,188,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,188,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'EQ_EQ':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,189,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,189,189,189,189,-226,-227,-228,-216,189,189,189,189,189,189,189,189,189,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,189,-246,189,-225,-230,-243,189,189,189,189,189,-245,-225,-249,-250,-252,-258,189,189,-15,189,189,-247,-271,-248,-272,-251,-274,189,-257,-280,189,189,-269,-270,-273,-254,-256,-279,-260,-262,-264,189,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,189,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'NOT_EQ':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,190,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,190,190,190,190,-226,-227,-228,-216,190,190,190,190,190,190,190,190,190,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,190,-246,190,-225,-230,-243,190,190,190,190,190,-245,-225,-249,-250,-252,-258,190,190,-15,190,190,-247,-271,-248,-272,-251,-274,190,-257,-280,190,190,-269,-270,-273,-254,-256,-279,-260,-262,-264,190,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,190,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'LT':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,191,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,191,191,191,191,-226,-227,-228,-216,191,191,191,191,191,191,191,191,191,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,191,-246,191,-225,-230,-243,191,191,191,191,191,-245,-225,-249,-250,-252,-258,191,191,-15,191,191,-247,-271,-248,-272,-251,-274,191,-257,-280,191,191,-269,-270,-273,-254,-256,-279,-260,-262,-264,191,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,191,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'LT_EQ':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,192,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,192,192,192,192,-226,-227,-228,-216,192,192,192,192,192,192,192,192,192,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,192,-246,192,-225,-230,-243,192,192,192,192,192,-245,-225,-249,-250,-252,-258,192,192,-15,192,192,-247,-271,-248,-272,-251,-274,192,-257,-280,192,192,-269,-270,-273,-254,-256,-279,-260,-262,-264,192,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,192,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'GT':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,193,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,193,193,193,193,-226,-227,-228,-216,193,193,193,193,193,193,193,193,193,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,193,-246,193,-225,-230,-243,193,193,193,193,193,-245,-225,-249,-250,-252,-258,193,193,-15,193,193,-247,-271,-248,-272,-251,-274,193,-257,-280,193,193,-269,-270,-273,-254,-256,-279,-260,-262,-264,193,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,193,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'GT_EQ':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,194,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,194,194,194,194,-226,-227,-228,-216,194,194,194,194,194,194,194,194,194,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,194,-246,194,-225,-230,-243,194,194,194,194,194,-245,-225,-249,-250,-252,-258,194,194,-15,194,194,-247,-271,-248,-272,-251,-274,194,-257,-280,194,194,-269,-270,-273,-254,-256,-279,-260,-262,-264,194,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,194,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'PLUS':([50,65,72,76,86,88,95,96,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,131,132,133,134,135,136,137,143,144,145,150,151,154,162,163,164,166,170,171,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,210,211,212,213,220,222,224,226,229,235,250,254,256,258,262,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,300,302,304,305,311,312,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,336,338,339,340,341,348,356,357,359,362,363,365,366,368,369,382,384,385,387,389,390,395,402,404,405,407,408,411,412,413,416,417,428,432,435,436,437,440,441,443,444,447,448,450,452,453,462,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,496,498,500,502,503,505,506,507,],[-292,113,113,113,113,113,-149,-291,-225,195,-195,-217,-218,-220,-221,-222,-215,113,-224,-219,-223,-229,113,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,113,113,195,113,195,113,113,195,113,195,113,-16,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,-226,-227,-228,113,113,-216,195,195,195,113,195,195,195,195,113,113,113,113,195,195,195,195,195,195,195,195,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,195,113,-246,195,-225,-230,-243,113,195,113,113,113,113,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,195,113,113,195,195,195,-245,-225,-249,-250,113,-252,113,-258,113,195,113,195,-15,195,195,113,-247,-271,-248,-272,-251,-274,195,-257,-280,195,195,113,-269,-270,-273,-254,113,-256,-279,-260,113,-262,-264,113,113,195,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,195,-275,-277,-281,-283,-285,-266,113,-268,-288,-265,-290,-267,-287,-289,]),'MINUS':([50,65,72,76,86,88,95,96,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,131,132,133,134,135,136,137,143,144,145,150,151,154,162,163,164,166,170,171,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,210,211,212,213,220,222,224,226,229,235,250,254,256,258,262,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,300,302,304,305,311,312,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,336,338,339,340,341,348,356,357,359,362,363,365,366,368,369,382,384,385,387,389,390,395,402,404,405,407,408,411,412,413,416,417,428,432,435,436,437,440,441,443,444,447,448,450,452,453,462,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,496,498,500,502,503,505,506,507,],[-292,114,114,114,114,114,-149,-291,-225,196,-195,-217,-218,-220,-221,-222,-215,114,-224,-219,-223,-229,114,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,114,114,196,114,196,114,114,196,114,196,114,-16,114,114,114,114,114,114,114,114,114,114,114,114,114,114,114,114,114,114,114,-226,-227,-228,114,114,-216,196,196,196,114,196,196,196,196,114,114,114,114,196,196,196,196,196,196,196,196,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,196,114,-246,196,-225,-230,-243,114,196,114,114,114,114,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,196,114,114,196,196,196,-245,-225,-249,-250,114,-252,114,-258,114,196,114,196,-15,196,196,114,-247,-271,-248,-272,-251,-274,196,-257,-280,196,196,114,-269,-270,-273,-254,114,-256,-279,-260,114,-262,-264,114,114,196,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,196,-275,-277,-281,-283,-285,-266,114,-268,-288,-265,-290,-267,-287,-289,]),'OR':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,197,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,197,197,197,197,-226,-227,-228,-216,197,197,197,197,197,197,197,197,197,197,197,197,197,197,197,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,197,-246,197,-225,-230,-243,197,197,197,197,197,-245,-225,-249,-250,-252,-258,197,197,-15,197,197,-247,-271,-248,-272,-251,-274,197,-257,-280,197,197,-269,-270,-273,-254,-256,-279,-260,-262,-264,197,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,197,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'CARET':([50,65,72,76,86,88,95,96,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,131,132,133,134,135,136,137,143,144,145,150,151,154,162,163,164,166,170,171,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,210,211,212,213,220,222,224,226,229,235,250,254,256,258,262,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,300,302,304,305,311,312,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,336,338,339,340,341,348,356,357,359,362,363,365,366,368,369,382,384,385,387,389,390,395,402,404,405,407,408,411,412,413,416,417,428,432,435,436,437,440,441,443,444,447,448,450,452,453,462,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,496,498,500,502,503,505,506,507,],[-292,115,115,115,115,115,-149,-291,-225,198,-195,-217,-218,-220,-221,-222,-215,115,-224,-219,-223,-229,115,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,115,115,198,115,198,115,115,198,115,198,115,-16,115,115,115,115,115,115,115,115,115,115,115,115,115,115,115,115,115,115,115,-226,-227,-228,115,115,-216,198,198,198,115,198,198,198,198,115,115,115,115,198,198,198,198,198,198,198,198,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,198,115,-246,198,-225,-230,-243,115,198,115,115,115,115,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,198,115,115,198,198,198,-245,-225,-249,-250,115,-252,115,-258,115,198,115,198,-15,198,198,115,-247,-271,-248,-272,-251,-274,198,-257,-280,198,198,115,-269,-270,-273,-254,115,-256,-279,-260,115,-262,-264,115,115,198,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,198,-275,-277,-281,-283,-285,-266,115,-268,-288,-265,-290,-267,-287,-289,]),'STAR':([50,65,72,76,86,88,95,96,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,131,132,133,134,135,136,137,138,143,144,145,150,151,154,162,163,164,166,170,171,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,210,211,212,213,215,216,220,222,224,226,229,235,250,254,256,258,262,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,300,302,304,305,311,312,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,336,338,339,340,341,348,356,357,359,362,363,365,366,368,369,372,382,384,385,387,389,390,395,402,404,405,407,408,411,412,413,416,417,428,432,435,436,437,440,441,443,444,447,448,450,452,453,462,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,496,498,500,502,503,505,506,507,],[-292,116,116,116,116,116,-149,-291,-225,199,-195,-217,-218,-220,-221,-222,-215,116,-224,-219,-223,-229,116,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-291,116,116,199,116,199,116,116,199,116,199,116,-16,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,-226,-227,-228,116,116,-216,199,310,-90,199,199,116,199,199,199,199,116,116,116,116,199,199,199,199,199,199,199,199,199,199,199,199,-208,-209,-210,-211,-212,-213,-214,-244,199,116,-246,199,-225,-230,-243,116,199,116,116,116,116,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,199,116,116,199,199,199,-245,-225,-249,-250,116,-252,116,-258,116,-91,199,116,199,-15,199,199,116,-247,-271,-248,-272,-251,-274,199,-257,-280,199,199,116,-269,-270,-273,-254,116,-256,-279,-260,116,-262,-264,116,116,199,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,199,-275,-277,-281,-283,-285,-266,116,-268,-288,-265,-290,-267,-287,-289,]),'DIVIDE':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,200,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,200,200,200,200,-226,-227,-228,-216,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,-208,-209,-210,-211,-212,-213,-214,-244,200,-246,200,-225,-230,-243,200,200,200,200,200,-245,-225,-249,-250,-252,-258,200,200,-15,200,200,-247,-271,-248,-272,-251,-274,200,-257,-280,200,200,-269,-270,-273,-254,-256,-279,-260,-262,-264,200,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,200,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'MODULO':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,201,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,201,201,201,201,-226,-227,-228,-216,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,-208,-209,-210,-211,-212,-213,-214,-244,201,-246,201,-225,-230,-243,201,201,201,201,201,-245,-225,-249,-250,-252,-258,201,201,-15,201,201,-247,-271,-248,-272,-251,-274,201,-257,-280,201,201,-269,-270,-273,-254,-256,-279,-260,-262,-264,201,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,201,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'LS':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,202,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,202,202,202,202,-226,-227,-228,-216,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,-208,-209,-210,-211,-212,-213,-214,-244,202,-246,202,-225,-230,-243,202,202,202,202,202,-245,-225,-249,-250,-252,-258,202,202,-15,202,202,-247,-271,-248,-272,-251,-274,202,-257,-280,202,202,-269,-270,-273,-254,-256,-279,-260,-262,-264,202,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,202,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'RS':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,203,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,203,203,203,203,-226,-227,-228,-216,203,203,203,203,203,203,203,203,203,203,203,203,203,203,203,203,203,203,203,-208,-209,-210,-211,-212,-213,-214,-244,203,-246,203,-225,-230,-243,203,203,203,203,203,-245,-225,-249,-250,-252,-258,203,203,-15,203,203,-247,-271,-248,-272,-251,-274,203,-257,-280,203,203,-269,-270,-273,-254,-256,-279,-260,-262,-264,203,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,203,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'AMP':([50,65,72,76,86,88,95,96,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,131,132,133,134,135,136,137,143,144,145,150,151,154,162,163,164,166,170,171,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,210,211,212,213,220,222,224,226,229,235,250,254,256,258,262,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,300,302,304,305,311,312,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,336,338,339,340,341,348,356,357,359,362,363,365,366,368,369,382,384,385,387,389,390,395,402,404,405,407,408,411,412,413,416,417,428,432,435,436,437,440,441,443,444,447,448,450,452,453,462,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,496,498,500,502,503,505,506,507,],[-292,117,117,117,117,117,-149,-291,-225,204,-195,-217,-218,-220,-221,-222,-215,117,-224,-219,-223,-229,117,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,117,117,204,117,204,117,117,204,117,204,117,-16,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,-226,-227,-228,117,117,-216,204,204,204,117,204,204,204,204,117,117,117,117,204,204,204,204,204,204,204,204,204,204,204,204,-208,-209,-210,-211,-212,-213,-214,-244,204,117,-246,204,-225,-230,-243,117,204,117,117,117,117,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,204,117,117,204,204,204,-245,-225,-249,-250,117,-252,117,-258,117,204,117,204,-15,204,204,117,-247,-271,-248,-272,-251,-274,204,-257,-280,204,204,117,-269,-270,-273,-254,117,-256,-279,-260,117,-262,-264,117,117,204,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,204,-275,-277,-281,-283,-285,-266,117,-268,-288,-265,-290,-267,-287,-289,]),'AND_OR':([50,95,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,145,151,163,166,206,207,208,212,213,220,222,226,229,235,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,300,302,304,305,312,336,340,341,348,356,357,359,362,365,368,382,385,387,389,390,402,404,405,407,408,411,412,413,416,417,428,435,436,437,440,443,444,447,450,452,464,466,468,469,471,472,474,475,477,478,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,205,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,205,205,205,205,-226,-227,-228,-216,205,205,205,205,205,205,205,205,205,205,205,205,205,205,205,205,205,205,205,-208,-209,-210,-211,-212,-213,-214,-244,205,-246,205,-225,-230,-243,205,205,205,205,205,-245,-225,-249,-250,-252,-258,205,205,-15,205,205,-247,-271,-248,-272,-251,-274,205,-257,-280,205,205,-269,-270,-273,-254,-256,-279,-260,-262,-264,205,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,205,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'RSQUARE':([50,95,109,110,111,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,298,304,305,356,359,362,365,368,387,402,404,405,407,408,411,413,416,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,186,-225,-88,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,356,-246,-230,-243,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'ASSIGN_OP':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,255,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,312,313,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,321,339,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-61,-59,321,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'PLUS_PLUS':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,322,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,322,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'MINUS_MINUS':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,323,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,323,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'PLUS_EQ':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,255,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,312,313,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,325,325,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-61,-59,325,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'MINUS_EQ':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,255,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,312,313,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,326,326,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-61,-59,326,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'OR_EQ':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,255,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,312,313,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,327,327,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-61,-59,327,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'CARET_EQ':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,255,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,312,313,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,328,328,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-61,-59,328,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'STAR_EQ':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,255,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,312,313,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,329,329,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-61,-59,329,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'DIVIDE_EQ':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,255,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,312,313,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,330,330,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-61,-59,330,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'MODULO_EQ':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,255,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,312,313,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,331,331,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-61,-59,331,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'LS_EQ':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,255,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,312,313,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,332,332,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-61,-59,332,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'RS_EQ':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,255,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,312,313,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,333,333,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-61,-59,333,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'AMP_EQ':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,255,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,312,313,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,334,334,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-61,-59,334,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'AND_OR_EQ':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,250,255,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,312,313,348,356,359,362,365,368,387,402,404,405,407,408,411,413,416,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,335,335,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-61,-59,335,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'DDD':([50,57,58,59,60,61,62,63,64,66,91,92,95,97,98,103,104,105,106,107,108,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,147,148,168,173,177,178,179,186,206,207,208,211,212,267,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,299,300,301,302,304,305,306,312,313,314,350,356,359,362,365,368,371,387,401,402,404,405,407,408,410,411,412,413,415,416,417,418,425,435,436,437,440,443,444,447,450,452,466,468,469,471,472,474,475,477,478,480,481,482,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-75,102,-79,-104,-80,-81,-82,-83,-84,-103,102,-149,176,-110,183,-106,-105,-107,-108,102,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-58,-56,176,-76,-77,-111,102,-291,-226,-227,-228,-291,-216,-112,-109,-85,-86,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,358,361,364,367,-230,-243,-89,-61,-59,-57,-78,-245,-249,-250,-252,-258,-103,-15,-87,-247,-271,-248,-272,-251,439,-274,442,-257,446,-280,449,451,-60,-269,-270,-273,-254,-256,-279,-260,-262,-264,-276,-253,-278,-255,-282,-259,-284,-261,-263,494,-286,497,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'COLON':([50,95,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,206,207,208,212,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,312,313,356,359,362,365,368,387,402,404,405,407,408,411,413,416,425,431,433,435,436,437,440,443,444,447,450,452,463,464,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,-149,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,-226,-227,-228,-216,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-61,-59,-245,-249,-250,-252,-258,-15,-247,-271,-248,-272,-251,-274,-257,-280,-60,462,-185,-269,-270,-273,-254,-256,-279,-260,-262,-264,-184,-186,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'LCURLY':([50,56,57,59,60,61,62,63,64,66,67,91,93,95,96,98,104,105,106,107,110,112,118,120,123,125,126,127,128,129,131,132,133,134,135,136,137,170,171,173,177,178,186,206,207,208,212,214,258,262,267,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,298,304,305,306,319,340,341,348,350,356,359,362,365,368,371,387,395,401,402,404,405,407,408,411,413,416,426,428,435,436,437,440,443,444,447,450,452,462,466,468,469,471,472,474,475,477,478,481,488,489,490,491,492,495,498,500,502,503,505,506,507,],[-292,96,-75,-79,-104,-80,-81,-82,-83,-84,138,-103,96,-149,-291,-110,-106,-105,-107,-108,-225,-195,-215,-224,-229,-231,-232,-233,-234,-235,-236,-237,-238,-242,-239,-240,-241,96,-16,-76,-77,-111,-291,-226,-227,-228,-216,96,96,347,-112,-109,-85,-86,-196,-197,-198,-199,-200,-201,-202,-203,-204,-205,-206,-207,-208,-209,-210,-211,-212,-213,-214,-244,-246,-230,-243,-89,96,96,96,398,-78,-245,-249,-250,-252,-258,96,-15,427,-87,-247,-271,-248,-272,-251,-274,-257,-280,96,461,-269,-270,-273,-254,-256,-279,-260,-262,-264,96,-276,-253,-278,-255,-282,-259,-284,-261,-263,-286,-275,-277,-281,-283,-285,-266,-268,-288,-265,-290,-267,-287,-289,]),'NOT':([65,72,76,86,88,96,113,114,115,116,117,119,121,122,124,143,144,150,154,162,164,170,171,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,338,339,363,366,369,384,395,432,441,448,453,462,496,],[121,121,121,121,121,-291,-217,-218,-220,-221,-222,121,-219,-223,121,121,121,121,121,121,121,121,-16,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,121,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,121,121,121,121,121,121,121,121,121,121,121,121,121,]),'LT_MINUS':([65,72,76,86,88,96,113,114,115,116,117,119,121,122,124,143,144,150,154,162,164,170,171,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,338,339,363,366,369,384,395,432,441,448,453,462,496,],[122,122,122,122,122,-291,-217,-218,-220,-221,-222,122,-219,-223,122,122,122,122,122,122,122,122,-16,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,122,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,122,122,122,122,122,122,122,122,122,122,122,122,122,]),'FLOAT_LIT':([65,72,76,86,88,96,113,114,115,116,117,119,121,122,124,143,144,150,154,162,164,170,171,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,338,339,363,366,369,384,395,432,441,448,453,462,496,],[134,134,134,134,134,-291,-217,-218,-220,-221,-222,134,-219,-223,134,134,134,134,134,134,134,134,-16,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,134,134,134,134,134,134,134,134,134,134,134,134,134,]),'DECIMAL_LIT':([65,72,76,86,88,96,113,114,115,116,117,119,121,122,124,143,144,150,154,162,164,170,171,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,338,339,363,366,369,384,395,432,441,448,453,462,496,],[135,135,135,135,135,-291,-217,-218,-220,-221,-222,135,-219,-223,135,135,135,135,135,135,135,135,-16,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,135,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,135,135,135,135,135,135,135,135,135,135,135,135,135,]),'OCTAL_LIT':([65,72,76,86,88,96,113,114,115,116,117,119,121,122,124,143,144,150,154,162,164,170,171,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,338,339,363,366,369,384,395,432,441,448,453,462,496,],[136,136,136,136,136,-291,-217,-218,-220,-221,-222,136,-219,-223,136,136,136,136,136,136,136,136,-16,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,136,136,136,136,136,136,136,136,136,136,136,136,136,]),'HEX_LIT':([65,72,76,86,88,96,113,114,115,116,117,119,121,122,124,143,144,150,154,162,164,170,171,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,324,325,326,327,328,329,330,331,332,333,334,335,338,339,363,366,369,384,395,432,441,448,453,462,496,],[137,137,137,137,137,-291,-217,-218,-220,-221,-222,137,-219,-223,137,137,137,137,137,137,137,137,-16,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,137,-160,-161,-162,-163,-164,-165,-166,-167,-168,-169,-170,-171,137,137,137,137,137,137,137,137,137,137,137,137,137,]),'RETURN':([96,170,171,319,462,],[-291,254,-16,254,254,]),'IF':([96,170,171,319,426,462,],[-291,256,-16,256,256,256,]),'FOR':([96,170,171,319,462,],[-291,258,-16,258,258,]),'BREAK':([96,170,171,319,462,],[-291,259,-16,259,259,]),'CONTINUE':([96,170,171,319,462,],[-291,260,-16,260,260,]),'GOTO':([96,170,171,319,462,],[-291,261,-16,261,261,]),'SWITCH':([96,170,171,319,462,],[-291,262,-16,262,262,]),'RCURLY':([96,138,170,171,215,216,237,239,317,318,319,347,372,388,396,397,398,427,430,434,460,461,462,486,487,],[-291,-291,-291,-16,306,-90,-291,-19,387,-17,-291,-291,-91,-18,429,-181,-291,-291,-182,465,485,-291,-291,499,-183,]),'CASE':([239,319,347,388,396,397,398,427,430,434,460,461,462,486,487,],[-19,-291,-291,-18,432,-181,-291,-291,-182,432,432,-291,-291,432,-183,]),'DEFAULT':([239,319,347,388,396,397,398,427,430,434,460,461,462,486,487,],[-19,-291,-291,-18,433,-181,-291,-291,-182,433,433,-291,-291,433,-183,]),'ELSE':([387,393,],[-15,426,]),}

_lr_action = {}
for _k, _v in _lr_action_items.items():
   for _x,_y in zip(_v[0],_v[1]):
      if not _x in _lr_action:  _lr_action[_x] = {}
      _lr_action[_x][_k] = _y
del _lr_action_items

_lr_goto_items = {'SourceFile':([0,],[1,]),'ImportDeclList':([4,21,],[5,45,]),'ImportDecl':([4,21,],[6,6,]),'empty':([4,5,8,21,22,27,28,30,32,33,36,39,42,43,44,58,60,74,81,90,96,97,102,103,108,138,141,168,170,176,183,186,211,237,297,308,309,319,347,373,374,381,398,422,427,454,456,461,462,],[7,11,26,7,48,11,59,69,59,59,79,59,84,59,59,59,59,59,59,48,171,59,59,59,59,216,59,59,239,59,59,59,59,318,59,59,378,239,397,378,378,378,397,378,397,378,378,397,239,]),'TopLevelDeclList':([5,27,],[9,53,]),'TopLevelDecl':([5,27,],[10,10,]),'Declaration':([5,27,170,319,462,],[12,12,240,240,240,]),'FunctionDecl':([5,27,],[13,13,]),'ConstDecl':([5,27,170,319,462,],[14,14,14,14,14,]),'TypeDecl':([5,27,170,319,462,],[15,15,15,15,15,]),'VarDecl':([5,27,170,319,462,],[16,16,16,16,16,]),'ImportSpec':([8,22,90,],[23,47,47,]),'FunctionName':([17,],[28,]),'ConstSpec':([18,68,],[31,140,]),'IdentifierList':([18,20,58,68,83,92,108,179,215,],[33,44,103,33,44,103,103,103,308,]),'TypeSpecTopList':([19,],[34,]),'TypeSpec':([19,78,],[35,156,]),'AliasDecl':([19,78,],[37,37,]),'TypeDef':([19,78,],[38,38,]),'VarSpecTopList':([20,],[40,]),'VarSpec':([20,83,],[41,161,]),'ImportSpecList':([22,90,],[46,167,]),'string_lit':([24,25,26,48,65,72,76,86,88,119,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,309,311,315,319,320,321,338,339,363,366,369,373,374,381,384,395,422,432,441,448,453,454,456,462,496,],[49,51,52,52,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,129,380,129,129,129,129,129,129,129,129,129,129,380,380,380,129,129,380,129,129,129,129,380,380,129,129,]),'Type':([28,32,33,39,43,44,58,60,74,81,97,102,103,108,141,168,176,183,186,211,297,308,],[55,71,75,82,85,87,99,106,99,159,175,180,182,99,71,175,266,270,274,301,99,373,]),'Signature':([28,54,55,130,303,],[56,91,93,214,371,]),'TypeLit':([28,32,33,39,43,44,58,60,74,81,97,102,103,108,141,168,176,183,186,211,297,308,],[57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,]),'Parameters':([28,54,55,60,130,303,],[60,60,60,104,60,60,]),'ArrayType':([28,32,33,39,43,44,58,60,74,81,97,102,103,108,141,168,176,183,186,211,297,308,],[61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,]),'StandardType':([28,32,33,39,43,44,58,60,74,81,97,102,103,108,141,168,176,183,186,211,297,308,],[62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,]),'StructType':([28,32,33,39,43,44,58,60,74,81,97,102,103,108,141,168,176,183,186,211,297,308,],[63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,]),'FunctionType':([28,32,33,39,43,44,58,60,74,81,97,102,103,108,141,168,176,183,186,211,297,308,],[64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,]),'ConstSpecList':([30,],[68,]),'TypeSpecList':([36,],[78,]),'VarSpecList':([42,],[83,]),'FunctionBody':([56,93,214,371,],[94,169,305,305,]),'Block':([56,93,170,214,258,319,340,341,371,426,462,],[95,95,243,95,342,243,393,394,95,457,243,]),'ParameterList':([58,92,108,],[100,100,100,]),'ParameterDecl':([58,92,108,179,],[101,101,101,268,]),'Result':([60,],[105,]),'ArrayLength':([65,],[109,]),'Expression':([65,72,76,86,88,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,338,339,363,366,369,384,395,432,441,448,453,462,496,],[111,145,151,163,166,213,220,222,226,229,235,166,250,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,296,300,312,336,340,341,348,213,382,385,250,389,390,166,166,312,412,417,312,428,464,312,312,482,250,312,]),'UnaryExpr':([65,72,76,86,88,119,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,338,339,363,366,369,384,395,432,441,448,453,462,496,],[112,112,112,112,112,212,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,112,]),'PrimaryExpr':([65,72,76,86,88,119,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,338,339,363,366,369,384,395,432,441,448,453,462,496,],[118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,]),'unary_op':([65,72,76,86,88,119,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,338,339,363,366,369,384,395,432,441,448,453,462,496,],[119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,119,]),'Operand':([65,72,76,86,88,119,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,338,339,363,366,369,384,395,432,441,448,453,462,496,],[120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,]),'Literal':([65,72,76,86,88,119,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,338,339,363,366,369,384,395,432,441,448,453,462,496,],[123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,]),'BasicLit':([65,72,76,86,88,119,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,338,339,363,366,369,384,395,432,441,448,453,462,496,],[125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,]),'FunctionLit':([65,72,76,86,88,119,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,338,339,363,366,369,384,395,432,441,448,453,462,496,],[126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,]),'int_lit':([65,72,76,86,88,119,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,338,339,363,366,369,384,395,432,441,448,453,462,496,],[127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,]),'float_lit':([65,72,76,86,88,119,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,338,339,363,366,369,384,395,432,441,448,453,462,496,],[128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,]),'decimal_lit':([65,72,76,86,88,119,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,338,339,363,366,369,384,395,432,441,448,453,462,496,],[131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,]),'octal_lit':([65,72,76,86,88,119,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,338,339,363,366,369,384,395,432,441,448,453,462,496,],[132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,]),'hex_lit':([65,72,76,86,88,119,124,143,144,150,154,162,164,170,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,210,211,224,254,256,258,262,297,311,315,319,320,321,338,339,363,366,369,384,395,432,441,448,453,462,496,],[133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,]),'ExpressionList':([72,76,88,143,144,150,154,164,170,211,254,262,311,315,319,338,339,366,369,432,453,462,],[146,152,165,221,223,227,230,236,255,299,337,255,383,386,255,391,392,410,415,463,480,255,]),'IdentifierBotList':([73,225,],[148,314,]),'ScopeStart':([96,],[170,]),'Selector':([118,],[206,]),'Index':([118,],[207,]),'Arguments':([118,],[208,]),'FieldDeclList':([138,],[215,]),'StatementList':([170,319,462,],[237,388,487,]),'Statement':([170,319,462,],[238,238,238,]),'SimpleStmt':([170,262,319,462,],[241,346,241,241,]),'ReturnStmt':([170,319,462,],[242,242,242,]),'IfStmt':([170,319,426,462,],[244,244,459,244,]),'SwitchStmt':([170,319,462,],[245,245,245,]),'ForStmt':([170,319,462,],[246,246,246,]),'BreakStmt':([170,319,462,],[247,247,247,]),'ContinueStmt':([170,319,462,],[248,248,248,]),'GotoStmt':([170,319,462,],[249,249,249,]),'Assignment':([170,262,319,462,],[251,251,251,251,]),'ShortVarDecl':([170,262,319,462,],[252,252,252,252,]),'IncDecStmt':([170,262,319,462,],[253,253,253,253,]),'ExprSwitchStmt':([170,319,462,],[257,257,257,]),'FieldDecl':([215,],[307,]),'ExpressionBotList':([224,363,384,441,448,496,],[313,313,425,313,313,313,]),'ScopeEnd':([237,],[317,]),'assign_op':([250,255,348,],[320,338,320,]),'TagTop':([309,373,374,381,422,454,456,],[377,419,420,424,455,483,484,]),'Tag':([309,373,374,381,422,454,456,],[379,379,379,379,379,379,379,]),'ExprCaseClauseList':([347,398,427,461,],[396,434,460,486,]),'ExprCaseClause':([396,434,460,486,],[430,430,430,430,]),'ExprSwitchCase':([396,434,460,486,],[431,431,431,431,]),'elseTail':([426,],[458,]),}

_lr_goto = {}
for _k, _v in _lr_goto_items.items():
   for _x, _y in zip(_v[0], _v[1]):
       if not _x in _lr_goto: _lr_goto[_x] = {}
       _lr_goto[_x][_k] = _y
del _lr_goto_items
_lr_productions = [
  ("S' -> SourceFile","S'",1,None,None,None),
  ('SourceFile -> PACKAGE IDENTIFIER SEMICOLON ImportDeclList TopLevelDeclList','SourceFile',5,'p_SourceFile','parser.py',139),
  ('ImportDeclList -> ImportDecl SEMICOLON ImportDeclList','ImportDeclList',3,'p_ImportDeclList','parser.py',152),
  ('ImportDeclList -> empty','ImportDeclList',1,'p_ImportDeclList','parser.py',153),
  ('TopLevelDeclList -> TopLevelDecl SEMICOLON TopLevelDeclList','TopLevelDeclList',3,'p_TopLevelDeclList','parser.py',162),
  ('TopLevelDeclList -> empty','TopLevelDeclList',1,'p_TopLevelDeclList','parser.py',163),
  ('TopLevelDecl -> Declaration','TopLevelDecl',1,'p_TopLevelDecl','parser.py',176),
  ('TopLevelDecl -> FunctionDecl','TopLevelDecl',1,'p_TopLevelDecl','parser.py',177),
  ('ImportDecl -> IMPORT LROUND ImportSpecList RROUND','ImportDecl',4,'p_ImportDecl','parser.py',185),
  ('ImportDecl -> IMPORT ImportSpec','ImportDecl',2,'p_ImportDecl','parser.py',186),
  ('ImportSpecList -> ImportSpec SEMICOLON ImportSpecList','ImportSpecList',3,'p_ImportSpecList','parser.py',194),
  ('ImportSpecList -> empty','ImportSpecList',1,'p_ImportSpecList','parser.py',195),
  ('ImportSpec -> DOT string_lit','ImportSpec',2,'p_ImportSpec','parser.py',203),
  ('ImportSpec -> IDENTIFIER string_lit','ImportSpec',2,'p_ImportSpec','parser.py',204),
  ('ImportSpec -> empty string_lit','ImportSpec',2,'p_ImportSpec','parser.py',205),
  ('Block -> LCURLY ScopeStart StatementList ScopeEnd RCURLY','Block',5,'p_Block','parser.py',213),
  ('ScopeStart -> empty','ScopeStart',1,'p_ScopeStart','parser.py',222),
  ('ScopeEnd -> empty','ScopeEnd',1,'p_ScopeEnd','parser.py',230),
  ('StatementList -> Statement SEMICOLON StatementList','StatementList',3,'p_StatementList','parser.py',238),
  ('StatementList -> empty','StatementList',1,'p_StatementList','parser.py',239),
  ('Statement -> Declaration','Statement',1,'p_Statement','parser.py',251),
  ('Statement -> SimpleStmt','Statement',1,'p_Statement','parser.py',252),
  ('Statement -> ReturnStmt','Statement',1,'p_Statement','parser.py',253),
  ('Statement -> Block','Statement',1,'p_Statement','parser.py',254),
  ('Statement -> IfStmt','Statement',1,'p_Statement','parser.py',255),
  ('Statement -> SwitchStmt','Statement',1,'p_Statement','parser.py',256),
  ('Statement -> ForStmt','Statement',1,'p_Statement','parser.py',257),
  ('Statement -> BreakStmt','Statement',1,'p_Statement','parser.py',258),
  ('Statement -> ContinueStmt','Statement',1,'p_Statement','parser.py',259),
  ('Statement -> GotoStmt','Statement',1,'p_Statement','parser.py',260),
  ('Declaration -> ConstDecl','Declaration',1,'p_Declaration','parser.py',268),
  ('Declaration -> TypeDecl','Declaration',1,'p_Declaration','parser.py',269),
  ('Declaration -> VarDecl','Declaration',1,'p_Declaration','parser.py',270),
  ('ConstDecl -> CONST LROUND ConstSpecList RROUND','ConstDecl',4,'p_ConstDecl','parser.py',278),
  ('ConstDecl -> CONST ConstSpec','ConstDecl',2,'p_ConstDecl','parser.py',279),
  ('ConstDecl -> CONST IDENTIFIER','ConstDecl',2,'p_ConstDecl','parser.py',280),
  ('ConstSpecList -> empty','ConstSpecList',1,'p_ConstSpecList','parser.py',286),
  ('ConstSpecList -> ConstSpecList ConstSpec SEMICOLON','ConstSpecList',3,'p_ConstSpecList','parser.py',287),
  ('ConstSpecList -> ConstSpecList IDENTIFIER SEMICOLON','ConstSpecList',3,'p_ConstSpecList','parser.py',288),
  ('ConstSpec -> IdentifierList','ConstSpec',1,'p_ConstSpec','parser.py',294),
  ('ConstSpec -> IdentifierList Type EQ Expression','ConstSpec',4,'p_ConstSpec','parser.py',295),
  ('ConstSpec -> IDENTIFIER Type EQ Expression','ConstSpec',4,'p_ConstSpec','parser.py',296),
  ('ConstSpec -> IdentifierList Type EQ ExpressionList','ConstSpec',4,'p_ConstSpec','parser.py',297),
  ('ConstSpec -> IDENTIFIER Type EQ ExpressionList','ConstSpec',4,'p_ConstSpec','parser.py',298),
  ('ConstSpec -> IdentifierList IDENTIFIER DOT IDENTIFIER EQ Expression','ConstSpec',6,'p_ConstSpec','parser.py',300),
  ('ConstSpec -> IDENTIFIER IDENTIFIER DOT IDENTIFIER EQ Expression','ConstSpec',6,'p_ConstSpec','parser.py',301),
  ('ConstSpec -> IdentifierList IDENTIFIER DOT IDENTIFIER EQ ExpressionList','ConstSpec',6,'p_ConstSpec','parser.py',302),
  ('ConstSpec -> IDENTIFIER IDENTIFIER DOT IDENTIFIER EQ ExpressionList','ConstSpec',6,'p_ConstSpec','parser.py',303),
  ('ConstSpec -> IdentifierList IDENTIFIER EQ Expression','ConstSpec',4,'p_ConstSpec','parser.py',305),
  ('ConstSpec -> IDENTIFIER IDENTIFIER EQ Expression','ConstSpec',4,'p_ConstSpec','parser.py',306),
  ('ConstSpec -> IdentifierList IDENTIFIER EQ ExpressionList','ConstSpec',4,'p_ConstSpec','parser.py',307),
  ('ConstSpec -> IDENTIFIER IDENTIFIER EQ ExpressionList','ConstSpec',4,'p_ConstSpec','parser.py',308),
  ('ConstSpec -> IdentifierList EQ Expression','ConstSpec',3,'p_ConstSpec','parser.py',309),
  ('ConstSpec -> IDENTIFIER EQ Expression','ConstSpec',3,'p_ConstSpec','parser.py',310),
  ('ConstSpec -> IdentifierList EQ ExpressionList','ConstSpec',3,'p_ConstSpec','parser.py',311),
  ('ConstSpec -> IDENTIFIER EQ ExpressionList','ConstSpec',3,'p_ConstSpec','parser.py',312),
  ('IdentifierList -> IDENTIFIER COMMA IdentifierBotList','IdentifierList',3,'p_IdentifierList','parser.py',319),
  ('IdentifierBotList -> IDENTIFIER COMMA IdentifierBotList','IdentifierBotList',3,'p_IdentifierBotList','parser.py',326),
  ('IdentifierBotList -> IDENTIFIER','IdentifierBotList',1,'p_IdentifierBotList','parser.py',327),
  ('ExpressionList -> Expression COMMA ExpressionBotList','ExpressionList',3,'p_ExpressionList','parser.py',339),
  ('ExpressionBotList -> Expression COMMA ExpressionBotList','ExpressionBotList',3,'p_ExpressionBotList','parser.py',348),
  ('ExpressionBotList -> Expression','ExpressionBotList',1,'p_ExpressionBotList','parser.py',349),
  ('TypeDecl -> TYPE TypeSpecTopList','TypeDecl',2,'p_TypeDecl','parser.py',363),
  ('TypeSpecTopList -> TypeSpec','TypeSpecTopList',1,'p_TypeSpecTopList','parser.py',369),
  ('TypeSpecTopList -> LROUND TypeSpecList RROUND','TypeSpecTopList',3,'p_TypeSpecTopList','parser.py',370),
  ('TypeSpecList -> empty','TypeSpecList',1,'p_TypeSpecList','parser.py',376),
  ('TypeSpecList -> TypeSpecList TypeSpec SEMICOLON','TypeSpecList',3,'p_TypeSpecList','parser.py',377),
  ('TypeSpec -> AliasDecl','TypeSpec',1,'p_TypeSpec','parser.py',383),
  ('TypeSpec -> TypeDef','TypeSpec',1,'p_TypeSpec','parser.py',384),
  ('AliasDecl -> IDENTIFIER EQ Type','AliasDecl',3,'p_AliasDecl','parser.py',390),
  ('AliasDecl -> IDENTIFIER EQ IDENTIFIER DOT IDENTIFIER','AliasDecl',5,'p_AliasDecl','parser.py',391),
  ('AliasDecl -> IDENTIFIER EQ IDENTIFIER','AliasDecl',3,'p_AliasDecl','parser.py',392),
  ('TypeDef -> IDENTIFIER Type','TypeDef',2,'p_TypeDef','parser.py',398),
  ('TypeDef -> IDENTIFIER IDENTIFIER','TypeDef',2,'p_TypeDef','parser.py',399),
  ('TypeDef -> IDENTIFIER IDENTIFIER DOT IDENTIFIER','TypeDef',4,'p_TypeDef','parser.py',400),
  ('Type -> TypeLit','Type',1,'p_Type','parser.py',406),
  ('Type -> LROUND IDENTIFIER RROUND','Type',3,'p_Type','parser.py',407),
  ('Type -> LROUND Type RROUND','Type',3,'p_Type','parser.py',408),
  ('Type -> LROUND IDENTIFIER DOT IDENTIFIER RROUND','Type',5,'p_Type','parser.py',409),
  ('Type -> empty','Type',1,'p_Type','parser.py',410),
  ('TypeLit -> ArrayType','TypeLit',1,'p_TypeLit','parser.py',422),
  ('TypeLit -> StandardType','TypeLit',1,'p_TypeLit','parser.py',423),
  ('TypeLit -> StructType','TypeLit',1,'p_TypeLit','parser.py',424),
  ('TypeLit -> FunctionType','TypeLit',1,'p_TypeLit','parser.py',425),
  ('StandardType -> PREDEF_TYPE','StandardType',1,'p_StandardType','parser.py',431),
  ('ArrayType -> LSQUARE ArrayLength RSQUARE Type','ArrayType',4,'p_ArrayType','parser.py',439),
  ('ArrayType -> LSQUARE ArrayLength RSQUARE IDENTIFIER','ArrayType',4,'p_ArrayType','parser.py',440),
  ('ArrayType -> LSQUARE ArrayLength RSQUARE IDENTIFIER DOT IDENTIFIER','ArrayType',6,'p_ArrayType','parser.py',441),
  ('ArrayLength -> Expression','ArrayLength',1,'p_ArrayLength','parser.py',447),
  ('StructType -> STRUCT LCURLY FieldDeclList RCURLY','StructType',4,'p_StructType','parser.py',453),
  ('FieldDeclList -> empty','FieldDeclList',1,'p_FieldDeclList','parser.py',459),
  ('FieldDeclList -> FieldDeclList FieldDecl SEMICOLON','FieldDeclList',3,'p_FieldDeclList','parser.py',460),
  ('FieldDecl -> IdentifierList Type TagTop','FieldDecl',3,'p_FieldDecl','parser.py',466),
  ('FieldDecl -> IdentifierList IDENTIFIER TagTop','FieldDecl',3,'p_FieldDecl','parser.py',467),
  ('FieldDecl -> IDENTIFIER IDENTIFIER','FieldDecl',2,'p_FieldDecl','parser.py',468),
  ('FieldDecl -> IdentifierList IDENTIFIER DOT IDENTIFIER TagTop','FieldDecl',5,'p_FieldDecl','parser.py',469),
  ('FieldDecl -> STAR IDENTIFIER DOT IDENTIFIER TagTop','FieldDecl',5,'p_FieldDecl','parser.py',470),
  ('FieldDecl -> IDENTIFIER DOT IDENTIFIER TagTop','FieldDecl',4,'p_FieldDecl','parser.py',471),
  ('FieldDecl -> STAR IDENTIFIER TagTop','FieldDecl',3,'p_FieldDecl','parser.py',472),
  ('FieldDecl -> IDENTIFIER TagTop','FieldDecl',2,'p_FieldDecl','parser.py',473),
  ('TagTop -> empty','TagTop',1,'p_TagTop','parser.py',479),
  ('TagTop -> Tag','TagTop',1,'p_TagTop','parser.py',480),
  ('Tag -> string_lit','Tag',1,'p_Tag','parser.py',486),
  ('FunctionType -> FUNC Signature','FunctionType',2,'p_FunctionType','parser.py',492),
  ('Signature -> Parameters','Signature',1,'p_Signature','parser.py',504),
  ('Signature -> Parameters Result','Signature',2,'p_Signature','parser.py',505),
  ('Result -> Parameters','Result',1,'p_Result','parser.py',511),
  ('Result -> Type','Result',1,'p_Result','parser.py',512),
  ('Result -> IDENTIFIER','Result',1,'p_Result','parser.py',513),
  ('Result -> IDENTIFIER DOT IDENTIFIER','Result',3,'p_Result','parser.py',514),
  ('Parameters -> LROUND RROUND','Parameters',2,'p_Parameters','parser.py',520),
  ('Parameters -> LROUND ParameterList RROUND','Parameters',3,'p_Parameters','parser.py',521),
  ('Parameters -> LROUND ParameterList COMMA RROUND','Parameters',4,'p_Parameters','parser.py',522),
  ('ParameterList -> ParameterDecl','ParameterList',1,'p_ParameterList','parser.py',529),
  ('ParameterList -> ParameterList COMMA ParameterDecl','ParameterList',3,'p_ParameterList','parser.py',530),
  ('ParameterDecl -> DDD Type','ParameterDecl',2,'p_ParameterDecl','parser.py',537),
  ('ParameterDecl -> IdentifierList Type','ParameterDecl',2,'p_ParameterDecl','parser.py',538),
  ('ParameterDecl -> IdentifierList DDD Type','ParameterDecl',3,'p_ParameterDecl','parser.py',539),
  ('ParameterDecl -> IDENTIFIER Type','ParameterDecl',2,'p_ParameterDecl','parser.py',540),
  ('ParameterDecl -> IDENTIFIER DDD Type','ParameterDecl',3,'p_ParameterDecl','parser.py',541),
  ('ParameterDecl -> DDD IDENTIFIER','ParameterDecl',2,'p_ParameterDecl','parser.py',542),
  ('ParameterDecl -> IdentifierList IDENTIFIER','ParameterDecl',2,'p_ParameterDecl','parser.py',543),
  ('ParameterDecl -> IdentifierList DDD IDENTIFIER','ParameterDecl',3,'p_ParameterDecl','parser.py',544),
  ('ParameterDecl -> IDENTIFIER IDENTIFIER','ParameterDecl',2,'p_ParameterDecl','parser.py',545),
  ('ParameterDecl -> IDENTIFIER DDD IDENTIFIER','ParameterDecl',3,'p_ParameterDecl','parser.py',546),
  ('ParameterDecl -> DDD IDENTIFIER DOT IDENTIFIER','ParameterDecl',4,'p_ParameterDecl','parser.py',547),
  ('ParameterDecl -> IdentifierList IDENTIFIER DOT IDENTIFIER','ParameterDecl',4,'p_ParameterDecl','parser.py',548),
  ('ParameterDecl -> IdentifierList DDD IDENTIFIER DOT IDENTIFIER','ParameterDecl',5,'p_ParameterDecl','parser.py',549),
  ('ParameterDecl -> IDENTIFIER IDENTIFIER DOT IDENTIFIER','ParameterDecl',4,'p_ParameterDecl','parser.py',550),
  ('ParameterDecl -> IDENTIFIER DDD IDENTIFIER DOT IDENTIFIER','ParameterDecl',5,'p_ParameterDecl','parser.py',551),
  ('VarDecl -> VAR VarSpecTopList','VarDecl',2,'p_VarDecl','parser.py',557),
  ('VarSpecTopList -> VarSpec','VarSpecTopList',1,'p_VarSpecTopList','parser.py',566),
  ('VarSpecTopList -> LROUND VarSpecList RROUND','VarSpecTopList',3,'p_VarSpecTopList','parser.py',567),
  ('VarSpecList -> empty','VarSpecList',1,'p_VarSpecList','parser.py',579),
  ('VarSpecList -> VarSpecList VarSpec SEMICOLON','VarSpecList',3,'p_VarSpecList','parser.py',580),
  ('VarSpec -> IDENTIFIER Type','VarSpec',2,'p_VarSpec','parser.py',587),
  ('VarSpec -> IDENTIFIER EQ Expression','VarSpec',3,'p_VarSpec','parser.py',588),
  ('VarSpec -> IDENTIFIER Type EQ Expression','VarSpec',4,'p_VarSpec','parser.py',589),
  ('VarSpec -> IdentifierList Type','VarSpec',2,'p_VarSpec','parser.py',590),
  ('VarSpec -> IdentifierList EQ ExpressionList','VarSpec',3,'p_VarSpec','parser.py',591),
  ('VarSpec -> IdentifierList Type EQ ExpressionList','VarSpec',4,'p_VarSpec','parser.py',592),
  ('VarSpecMid -> empty','VarSpecMid',1,'p_VarSpecMid','parser.py',612),
  ('VarSpecMid -> EQ ExpressionList','VarSpecMid',2,'p_VarSpecMid','parser.py',613),
  ('VarSpecMid -> EQ Expression','VarSpecMid',2,'p_VarSpecMid','parser.py',614),
  ('FunctionDecl -> FUNC FunctionName Type Signature','FunctionDecl',4,'p_FunctionDecl','parser.py',620),
  ('FunctionDecl -> FUNC FunctionName Signature','FunctionDecl',3,'p_FunctionDecl','parser.py',621),
  ('FunctionDecl -> FUNC FunctionName Type Signature FunctionBody','FunctionDecl',5,'p_FunctionDecl','parser.py',622),
  ('FunctionDecl -> FUNC FunctionName Signature FunctionBody','FunctionDecl',4,'p_FunctionDecl','parser.py',623),
  ('FunctionName -> IDENTIFIER','FunctionName',1,'p_FunctionName','parser.py',637),
  ('FunctionBody -> Block','FunctionBody',1,'p_FunctionBody','parser.py',644),
  ('SimpleStmt -> Expression','SimpleStmt',1,'p_SimpleStmt','parser.py',652),
  ('SimpleStmt -> Assignment','SimpleStmt',1,'p_SimpleStmt','parser.py',653),
  ('SimpleStmt -> ShortVarDecl','SimpleStmt',1,'p_SimpleStmt','parser.py',654),
  ('SimpleStmt -> IncDecStmt','SimpleStmt',1,'p_SimpleStmt','parser.py',655),
  ('IncDecStmt -> Expression PLUS_PLUS','IncDecStmt',2,'p_IncDecStmt','parser.py',669),
  ('IncDecStmt -> Expression MINUS_MINUS','IncDecStmt',2,'p_IncDecStmt','parser.py',670),
  ('ShortVarDecl -> ExpressionList ASSIGN_OP ExpressionList','ShortVarDecl',3,'p_ShortVarDecl','parser.py',686),
  ('ShortVarDecl -> Expression ASSIGN_OP Expression','ShortVarDecl',3,'p_ShortVarDecl','parser.py',687),
  ('Assignment -> ExpressionList assign_op ExpressionList','Assignment',3,'p_Assignment','parser.py',734),
  ('Assignment -> Expression assign_op Expression','Assignment',3,'p_Assignment','parser.py',735),
  ('assign_op -> EQ','assign_op',1,'p_assign_op','parser.py',795),
  ('assign_op -> PLUS_EQ','assign_op',1,'p_assign_op','parser.py',796),
  ('assign_op -> MINUS_EQ','assign_op',1,'p_assign_op','parser.py',797),
  ('assign_op -> OR_EQ','assign_op',1,'p_assign_op','parser.py',798),
  ('assign_op -> CARET_EQ','assign_op',1,'p_assign_op','parser.py',799),
  ('assign_op -> STAR_EQ','assign_op',1,'p_assign_op','parser.py',800),
  ('assign_op -> DIVIDE_EQ','assign_op',1,'p_assign_op','parser.py',801),
  ('assign_op -> MODULO_EQ','assign_op',1,'p_assign_op','parser.py',802),
  ('assign_op -> LS_EQ','assign_op',1,'p_assign_op','parser.py',803),
  ('assign_op -> RS_EQ','assign_op',1,'p_assign_op','parser.py',804),
  ('assign_op -> AMP_EQ','assign_op',1,'p_assign_op','parser.py',805),
  ('assign_op -> AND_OR_EQ','assign_op',1,'p_assign_op','parser.py',806),
  ('IfStmt -> IF Expression Block','IfStmt',3,'p_IfStmt','parser.py',813),
  ('IfStmt -> IF Expression Block ELSE elseTail','IfStmt',5,'p_IfStmt','parser.py',814),
  ('elseTail -> IfStmt','elseTail',1,'p_elseTail','parser.py',839),
  ('elseTail -> Block','elseTail',1,'p_elseTail','parser.py',840),
  ('SwitchStmt -> ExprSwitchStmt','SwitchStmt',1,'p_SwitchStmt','parser.py',848),
  ('ExprSwitchStmt -> SWITCH SimpleStmt SEMICOLON LCURLY ExprCaseClauseList RCURLY','ExprSwitchStmt',6,'p_ExprSwitchStmt','parser.py',855),
  ('ExprSwitchStmt -> SWITCH SimpleStmt SEMICOLON Expression LCURLY ExprCaseClauseList RCURLY','ExprSwitchStmt',7,'p_ExprSwitchStmt','parser.py',856),
  ('ExprSwitchStmt -> SWITCH LCURLY ExprCaseClauseList RCURLY','ExprSwitchStmt',4,'p_ExprSwitchStmt','parser.py',857),
  ('ExprSwitchStmt -> SWITCH Expression LCURLY ExprCaseClauseList RCURLY','ExprSwitchStmt',5,'p_ExprSwitchStmt','parser.py',858),
  ('ExprCaseClauseList -> empty','ExprCaseClauseList',1,'p_ExprCaseClauseList','parser.py',887),
  ('ExprCaseClauseList -> ExprCaseClauseList ExprCaseClause','ExprCaseClauseList',2,'p_ExprCaseClauseList','parser.py',888),
  ('ExprCaseClause -> ExprSwitchCase COLON StatementList','ExprCaseClause',3,'p_ExprCaseClause','parser.py',909),
  ('ExprSwitchCase -> CASE ExpressionList','ExprSwitchCase',2,'p_ExprSwitchCase','parser.py',924),
  ('ExprSwitchCase -> DEFAULT','ExprSwitchCase',1,'p_ExprSwitchCase','parser.py',925),
  ('ExprSwitchCase -> CASE Expression','ExprSwitchCase',2,'p_ExprSwitchCase','parser.py',926),
  ('ForStmt -> FOR Expression Block','ForStmt',3,'p_ForStmt','parser.py',937),
  ('ForStmt -> FOR Block','ForStmt',2,'p_ForStmt','parser.py',938),
  ('ReturnStmt -> RETURN','ReturnStmt',1,'p_ReturnStmt','parser.py',962),
  ('ReturnStmt -> RETURN Expression','ReturnStmt',2,'p_ReturnStmt','parser.py',963),
  ('ReturnStmt -> RETURN ExpressionList','ReturnStmt',2,'p_ReturnStmt','parser.py',964),
  ('BreakStmt -> BREAK IDENTIFIER','BreakStmt',2,'p_BreakStmt','parser.py',979),
  ('ContinueStmt -> CONTINUE IDENTIFIER','ContinueStmt',2,'p_ContinueStmt','parser.py',985),
  ('GotoStmt -> GOTO IDENTIFIER','GotoStmt',2,'p_GotoStmt','parser.py',991),
  ('Expression -> UnaryExpr','Expression',1,'p_Expression','parser.py',997),
  ('Expression -> Expression OR_OR Expression','Expression',3,'p_Expression','parser.py',998),
  ('Expression -> Expression AMP_AMP Expression','Expression',3,'p_Expression','parser.py',999),
  ('Expression -> Expression EQ_EQ Expression','Expression',3,'p_Expression','parser.py',1000),
  ('Expression -> Expression NOT_EQ Expression','Expression',3,'p_Expression','parser.py',1001),
  ('Expression -> Expression LT Expression','Expression',3,'p_Expression','parser.py',1002),
  ('Expression -> Expression LT_EQ Expression','Expression',3,'p_Expression','parser.py',1003),
  ('Expression -> Expression GT Expression','Expression',3,'p_Expression','parser.py',1004),
  ('Expression -> Expression GT_EQ Expression','Expression',3,'p_Expression','parser.py',1005),
  ('Expression -> Expression PLUS Expression','Expression',3,'p_Expression','parser.py',1006),
  ('Expression -> Expression MINUS Expression','Expression',3,'p_Expression','parser.py',1007),
  ('Expression -> Expression OR Expression','Expression',3,'p_Expression','parser.py',1008),
  ('Expression -> Expression CARET Expression','Expression',3,'p_Expression','parser.py',1009),
  ('Expression -> Expression STAR Expression','Expression',3,'p_Expression','parser.py',1010),
  ('Expression -> Expression DIVIDE Expression','Expression',3,'p_Expression','parser.py',1011),
  ('Expression -> Expression MODULO Expression','Expression',3,'p_Expression','parser.py',1012),
  ('Expression -> Expression LS Expression','Expression',3,'p_Expression','parser.py',1013),
  ('Expression -> Expression RS Expression','Expression',3,'p_Expression','parser.py',1014),
  ('Expression -> Expression AMP Expression','Expression',3,'p_Expression','parser.py',1015),
  ('Expression -> Expression AND_OR Expression','Expression',3,'p_Expression','parser.py',1016),
  ('UnaryExpr -> PrimaryExpr','UnaryExpr',1,'p_UnaryExpr','parser.py',1048),
  ('UnaryExpr -> unary_op UnaryExpr','UnaryExpr',2,'p_UnaryExpr','parser.py',1049),
  ('unary_op -> PLUS','unary_op',1,'p_unary_op','parser.py',1068),
  ('unary_op -> MINUS','unary_op',1,'p_unary_op','parser.py',1069),
  ('unary_op -> NOT','unary_op',1,'p_unary_op','parser.py',1070),
  ('unary_op -> CARET','unary_op',1,'p_unary_op','parser.py',1071),
  ('unary_op -> STAR','unary_op',1,'p_unary_op','parser.py',1072),
  ('unary_op -> AMP','unary_op',1,'p_unary_op','parser.py',1073),
  ('unary_op -> LT_MINUS','unary_op',1,'p_unary_op','parser.py',1074),
  ('PrimaryExpr -> Operand','PrimaryExpr',1,'p_PrimaryExpr','parser.py',1081),
  ('PrimaryExpr -> IDENTIFIER','PrimaryExpr',1,'p_PrimaryExpr','parser.py',1082),
  ('PrimaryExpr -> PrimaryExpr Selector','PrimaryExpr',2,'p_PrimaryExpr','parser.py',1083),
  ('PrimaryExpr -> PrimaryExpr Index','PrimaryExpr',2,'p_PrimaryExpr','parser.py',1084),
  ('PrimaryExpr -> PrimaryExpr Arguments','PrimaryExpr',2,'p_PrimaryExpr','parser.py',1085),
  ('Operand -> Literal','Operand',1,'p_Operand','parser.py',1101),
  ('Operand -> LROUND Expression RROUND','Operand',3,'p_Operand','parser.py',1102),
  ('Literal -> BasicLit','Literal',1,'p_Literal','parser.py',1113),
  ('Literal -> FunctionLit','Literal',1,'p_Literal','parser.py',1114),
  ('BasicLit -> int_lit','BasicLit',1,'p_BasicLit','parser.py',1126),
  ('BasicLit -> float_lit','BasicLit',1,'p_BasicLit','parser.py',1127),
  ('BasicLit -> string_lit','BasicLit',1,'p_BasicLit','parser.py',1128),
  ('int_lit -> decimal_lit','int_lit',1,'p_int_lit','parser.py',1136),
  ('int_lit -> octal_lit','int_lit',1,'p_int_lit','parser.py',1137),
  ('int_lit -> hex_lit','int_lit',1,'p_int_lit','parser.py',1138),
  ('decimal_lit -> DECIMAL_LIT','decimal_lit',1,'p_decimal_lit','parser.py',1146),
  ('octal_lit -> OCTAL_LIT','octal_lit',1,'p_octal_lit','parser.py',1153),
  ('hex_lit -> HEX_LIT','hex_lit',1,'p_hex_lit','parser.py',1160),
  ('float_lit -> FLOAT_LIT','float_lit',1,'p_float_lit','parser.py',1167),
  ('FunctionLit -> FUNC Signature FunctionBody','FunctionLit',3,'p_FunctionLit','parser.py',1174),
  ('Selector -> DOT IDENTIFIER','Selector',2,'p_Selector','parser.py',1180),
  ('Index -> LSQUARE Expression RSQUARE','Index',3,'p_Index','parser.py',1186),
  ('Arguments -> LROUND RROUND','Arguments',2,'p_Arguments','parser.py',1192),
  ('Arguments -> LROUND ExpressionList DDD RROUND','Arguments',4,'p_Arguments','parser.py',1193),
  ('Arguments -> LROUND Expression DDD RROUND','Arguments',4,'p_Arguments','parser.py',1194),
  ('Arguments -> LROUND ExpressionList RROUND','Arguments',3,'p_Arguments','parser.py',1195),
  ('Arguments -> LROUND Expression RROUND','Arguments',3,'p_Arguments','parser.py',1196),
  ('Arguments -> LROUND Type DDD RROUND','Arguments',4,'p_Arguments','parser.py',1197),
  ('Arguments -> LROUND Type RROUND','Arguments',3,'p_Arguments','parser.py',1198),
  ('Arguments -> LROUND Type COMMA ExpressionList DDD RROUND','Arguments',6,'p_Arguments','parser.py',1199),
  ('Arguments -> LROUND Type COMMA ExpressionList RROUND','Arguments',5,'p_Arguments','parser.py',1200),
  ('Arguments -> LROUND Type COMMA Expression DDD RROUND','Arguments',6,'p_Arguments','parser.py',1201),
  ('Arguments -> LROUND Type COMMA Expression RROUND','Arguments',5,'p_Arguments','parser.py',1202),
  ('Arguments -> LROUND IDENTIFIER DDD RROUND','Arguments',4,'p_Arguments','parser.py',1203),
  ('Arguments -> LROUND IDENTIFIER RROUND','Arguments',3,'p_Arguments','parser.py',1204),
  ('Arguments -> LROUND IDENTIFIER COMMA ExpressionList DDD RROUND','Arguments',6,'p_Arguments','parser.py',1205),
  ('Arguments -> LROUND IDENTIFIER COMMA ExpressionList RROUND','Arguments',5,'p_Arguments','parser.py',1206),
  ('Arguments -> LROUND IDENTIFIER COMMA Expression DDD RROUND','Arguments',6,'p_Arguments','parser.py',1207),
  ('Arguments -> LROUND IDENTIFIER COMMA Expression RROUND','Arguments',5,'p_Arguments','parser.py',1208),
  ('Arguments -> LROUND IDENTIFIER DOT IDENTIFIER DDD RROUND','Arguments',6,'p_Arguments','parser.py',1209),
  ('Arguments -> LROUND IDENTIFIER DOT IDENTIFIER RROUND','Arguments',5,'p_Arguments','parser.py',1210),
  ('Arguments -> LROUND IDENTIFIER DOT IDENTIFIER COMMA ExpressionList DDD RROUND','Arguments',8,'p_Arguments','parser.py',1211),
  ('Arguments -> LROUND IDENTIFIER DOT IDENTIFIER COMMA ExpressionList RROUND','Arguments',7,'p_Arguments','parser.py',1212),
  ('Arguments -> LROUND IDENTIFIER DOT IDENTIFIER COMMA Expression DDD RROUND','Arguments',8,'p_Arguments','parser.py',1213),
  ('Arguments -> LROUND IDENTIFIER DOT IDENTIFIER COMMA Expression RROUND','Arguments',7,'p_Arguments','parser.py',1214),
  ('Arguments -> LROUND ExpressionList DDD COMMA RROUND','Arguments',5,'p_Arguments','parser.py',1215),
  ('Arguments -> LROUND Expression DDD COMMA RROUND','Arguments',5,'p_Arguments','parser.py',1216),
  ('Arguments -> LROUND ExpressionList COMMA RROUND','Arguments',4,'p_Arguments','parser.py',1217),
  ('Arguments -> LROUND Expression COMMA RROUND','Arguments',4,'p_Arguments','parser.py',1218),
  ('Arguments -> LROUND Type DDD COMMA RROUND','Arguments',5,'p_Arguments','parser.py',1219),
  ('Arguments -> LROUND Type COMMA RROUND','Arguments',4,'p_Arguments','parser.py',1220),
  ('Arguments -> LROUND Type COMMA ExpressionList DDD COMMA RROUND','Arguments',7,'p_Arguments','parser.py',1221),
  ('Arguments -> LROUND Type COMMA ExpressionList COMMA RROUND','Arguments',6,'p_Arguments','parser.py',1222),
  ('Arguments -> LROUND Type COMMA Expression DDD COMMA RROUND','Arguments',7,'p_Arguments','parser.py',1223),
  ('Arguments -> LROUND Type COMMA Expression COMMA RROUND','Arguments',6,'p_Arguments','parser.py',1224),
  ('Arguments -> LROUND IDENTIFIER DDD COMMA RROUND','Arguments',5,'p_Arguments','parser.py',1225),
  ('Arguments -> LROUND IDENTIFIER COMMA RROUND','Arguments',4,'p_Arguments','parser.py',1226),
  ('Arguments -> LROUND IDENTIFIER COMMA ExpressionList DDD COMMA RROUND','Arguments',7,'p_Arguments','parser.py',1227),
  ('Arguments -> LROUND IDENTIFIER COMMA ExpressionList COMMA RROUND','Arguments',6,'p_Arguments','parser.py',1228),
  ('Arguments -> LROUND IDENTIFIER COMMA Expression DDD COMMA RROUND','Arguments',7,'p_Arguments','parser.py',1229),
  ('Arguments -> LROUND IDENTIFIER COMMA Expression COMMA RROUND','Arguments',6,'p_Arguments','parser.py',1230),
  ('Arguments -> LROUND IDENTIFIER DOT IDENTIFIER DDD COMMA RROUND','Arguments',7,'p_Arguments','parser.py',1231),
  ('Arguments -> LROUND IDENTIFIER DOT IDENTIFIER COMMA RROUND','Arguments',6,'p_Arguments','parser.py',1232),
  ('Arguments -> LROUND IDENTIFIER DOT IDENTIFIER COMMA ExpressionList DDD COMMA RROUND','Arguments',9,'p_Arguments','parser.py',1233),
  ('Arguments -> LROUND IDENTIFIER DOT IDENTIFIER COMMA ExpressionList COMMA RROUND','Arguments',8,'p_Arguments','parser.py',1234),
  ('Arguments -> LROUND IDENTIFIER DOT IDENTIFIER COMMA Expression DDD COMMA RROUND','Arguments',9,'p_Arguments','parser.py',1235),
  ('Arguments -> LROUND IDENTIFIER DOT IDENTIFIER COMMA Expression COMMA RROUND','Arguments',8,'p_Arguments','parser.py',1236),
  ('empty -> <empty>','empty',0,'p_empty','parser.py',1243),
  ('string_lit -> STRING_LIT','string_lit',1,'p_string_lit','parser.py',1248),
]

# Referencias

- Aho, A. V., & Aho, A. V. (Eds.). (2007). Compilers: Principles, techniques, & 
tools (2nd ed). Pearson/Addison Wesley.
- Aho, A. V., Sethi, R., & Ullman, J. D. (1986). Compilers, principles, techniques, and tools. Addison-Wesley Pub. Co.
- Codewalk: First-class functions in go - the go programming language. (n.d.). - Retrieved 3 December 2022, from https://go.dev/doc/codewalk/functions/
- Go—Operators precedence. (n.d.). Retrieved 3 December 2022, from https://www.tutorialspoint.com/go/go_operators_precedence.htm
- PLY (Python lex-yacc)—Ply 4.0 documentation. (n.d.). Retrieved 3 December 2022, from https://ply.readthedocs.io/en/latest/
- The go programming language specification—The go programming language. (n.d.). Retrieved 3 December 2022, from https://go.dev/ref/spec
- Three address code in Compiler. (2018, May 21). GeeksforGeeks. https://www.geeksforgeeks.org/three-address-code-compiler/


